# Construct of the ARGs Database
This code will process the ARG data from UNIPROT, CARD and ARDB to generate a consensus database:



## Remove duplicates

UNIPROT databases and metadata are stored in ./fasta directory. Then, card+uniprot+ardb+snps are merged into one big fasta file (ARGS.fasta)

ARGS.fasta is submited to CD-HIT webservice and genes are clustered (100% identity). Results are stored into the ./clusters directory

clusters is the output from cd-hit

In [ ]:
awk '{if($0~/>Cluster/){x=$2}else{gsub(">","",$3); gsub(/\.\.\./,"",$3); gsub(/\*/,"1",$4); gsub(/at/,"0",$4); split($3,y,"|"); gsub("aa,","",$2); gsub("%","",$5); print x"\t"y[1]"\t"y[2]"\t"$4}}' clusters > clusters.tsv

clusters.tsv is processed with process_clusters.py script to obtain the representative hit of each cluster that is from the card database. If not the longest one is obtained.

In [ ]:
python process_clusters.py > clusters.rep_hit.tsv

## Representative hits 

http://weizhongli-lab.org/cd-hit/

http://www.diamondsearch.org/index.php

Representative hits are clustered with cdhit to a 90% using the CD-Hit online version. (CLUSTERS.csv) then Uniprot sequences are aligned to the known ARGs (alignments.tsv) using diamond.

files were stored as json files using the functions under the utils.py file. Thus, three main files were generated

Metadata: contains the information of each gene, if the gene is from ARDB it contains its type and subtype.
Clusters: after removing duplicates, genes are clustered based on their identity. 
Alignments contains the alignments from diamond blastx of the UNIPROT agains the ARGdbs (CARD, ARDB, RESFAMS).    

(RESFAMS was not used because not enough metadata)

In [ ]:
import json
import utils
import cPickle
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


path = './db/'

metadata = json.load(open(path+'metadata_with_multilabels.json'))
clusters = json.load(open(path+'CLUSTERS.csv.json'))
alignments = json.load(open(path+'alignment.tsv.json'))

For each cluster, get the proteins and its best hit. Then apply the annotation rules to determine the quality of the classification ("High","Mid","Low", and "Manual Inspection")

In [ ]:
iden = 90

DB={}

for cluster in clusters:
    cluster_stats = utils.consensus_type(clusters[cluster], metadata, alignments)
    DB.update(utils.cluster_annotation(cluster_stats, 90, 1e-10, metadata))

I found out that the deep learning was giving me very high probabilities (>0.95) for some of the genes, but, the labels were completely different. After checking the best hits respect the CARD database I found that the CARD best hit corresponded to the predicted by the deep learning model. Thus, I decided to check the ARDB annotation and adjusted it to the CARD. This manual process ended up with improving the quality of the annotations and mislabels.

ARDB has misclassifications according to the CARD database, so, I need first to remove those mis-classifications. First, take all the ARDB genes from DB and check whether they have an identity >90% if so, assign the type and subtype of the ARDB gene as the CARD best Hit. 


In [ ]:
for i in DB:
    try:
        if DB[i]['database']=="ARDB":
            bhit = utils.best_hit(alignments[i], db="ARDB")[0]
            if float(bhit['identity'])>90:
                DB[i]['type'] = DB[bhit['subject']]['type']+"---"+DB[i]['type']
                DB[i]['subtype'] = DB[bhit['subject']]['subtype']
                DB[i]['bestHit'] = bhit['subject']
                DB[i]['evalue'] = bhit['evalue']
                DB[i]['iden'] = bhit['identity']
    except:
        pass

Chech how many number of sequences were classified according to their annotation quality. Then, Store the database.

In [ ]:
vari = 'AnFactor'
from collections import Counter
Counter([DB[i][vari] for i in DB])

json.dump(DB, open(path+"ANNOTATION_FULL2.json",'w'))

# Filter the database according to its quality

Load the full database

In [ ]:
import json
import utils

path = './db/'
DB = json.load(open(path+"ANNOTATION_FULL2.json"))

Setup a set of filters to get a clean and high quality database

In [ ]:
cnt = 0
NDB = {}

#Get the sequences that have the unknown - xxx transition or if they keep the same annotation through the best matched hit

for i in DB:
    cnd = [ai for ai in DB[i]['type'].split("---") if ai in DB[i]['prevType'].split("---")]
    if not cnd: # the previous type is not the current type
        if DB[i]['prevType']== 'unknown': # if there were not any annotation
        #print DB[i]
            NDB.update({i:DB[i]})
            cnt+=1
        elif DB[i]['AnFactor'] == 'High': # if the prev and current type differ but the new annotation is high
            NDB.update({i:DB[i]})
            cnt+=1
    else:
        NDB.update({i:DB[i]})
        cnt+=1

# 2 remove low quality annotation
NDB = {i:NDB[i] for i in NDB if NDB[i]['AnFactor']!="Low"}

# 3 remove manual-inspection genes
NDB = {i:NDB[i] for i in NDB if NDB[i]['AnFactor']!="Manual Inspection"}

# 4 remove genes that didin't get any anotation (no uniprot arg matches)
NDB = {i:NDB[i] for i in NDB if NDB[i]['database']!="UNIPROT-NO-MATCHES"}

Store filtered database in drive

In [ ]:
json.dump(NDB, open(path+"FilteredDatabase.json",'w'))

After looking at the database, we find out that some genes that were related to SNPS were still present, those entries were screened and removed from the dataset. The included genes are listed under the ./fasta/snp.fasta file.

List of manually corrected genes:

    A0A084IXL3	
    J9ASZ6	
    J8ZX71	
    R8CTG7	
    R8S2C7	
    A0A120EEM8	
    S3I4V9	
    J8NNE4	
    R8NA09	
    J8PHM1	
    J8EYN3	
    C2V7Y8	
    A0A0K0Q868	
    J8RR07	
    J7UNL3	
    A0A0J7EMG5	
    R8EI38	
    A0A0W7YGE7	
    A0A0A0WUC6	
    A0A0G8CQW8	
    J8DHH7	
    A0A0J7DCB6	
    R8DPN5	
    A0A0D7XKJ4	
    A0A0U1AMC0	

manually corrected subtypes:

    EMRK	multidrug
    ADER	multidrug
    ADES	multidrug
    MEPA	multidrug
    SUL1    sulfonamide
    MDFA    multidrug
    AAC(3)-IVA  aminoglycoside
    OPRA    multidrug
    OLEC    macrolide-lincosamide-streptogramin
    MDTL    multidrug
    EMRE    multidrug
    OPCM    multidrug
    SAT-4   aminoglycoside
    CEOA    multidrug
    ACRB    multidrug
    AAC(6')-IT  aminoglycoside
    YFBG    polymyxin

Then, ArGs were aligned again to search for duplicates that passed the previous filter. 

Duplicates from uniprot:

    entry_id	knownARG	identity	antibiotic-class	ARG-name
    A0A023PNW3 AAG45720.1 100.00 beta_lactam OXA-10
    B8ZHY2 CAA82916.1 100.00 beta_lactam OXY-1-1
    B8ZHY9 AAL79541.1 100.00 beta_lactam OXY-2-2
    L7T9C0 AEM44654.1 100.00 beta_lactam CTX-M-109
    T2B2C0 YP_008532725.1 100.00 beta_lactam KPC-2
    H6UQH2 CAD70280.1 100.00 beta_lactam CTX-M-28
    H6UQH3 ABN09669.1 100.00 beta_lactam CTX-M-61
    H6UQH0 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0A076V6C1 ABB72225.1 100.00 beta_lactam CTX-M-53
    H6UQH4 AFQ23954.1 100.00 beta_lactam SHV-148
    H6UQH5 AFQ23961.1 100.00 beta_lactam SHV-155
    A0A145MSF7 AHM76771.1 100.00 beta_lactam ACT-17
    U3PVF4 YP_008532725.1 100.00 beta_lactam KPC-2
    H6UQH9 AAF05311.2 100.00 beta_lactam CTX-M-9
    V9XVD2 BAM62793.1 100.00 beta_lactam IMP-42
    X2L3Y0 AIT76094.1 100.00 beta_lactam DHA-18
    J9V9J2 BAL68178.1 100.00 beta_lactam TEM-198
    I7AK64 AAD37413.1 100.00 beta_lactam SHV-2
    A0A0E3MUC8 ABO42050 100.00 kanamycin---dibekacin---tobramycin---sisomicin---gentamicin Ant2Ia
    A0A024A2N9 AEM44654.1 100.00 beta_lactam CTX-M-109
    G5DDS7 AFY98865.1 100.00 beta_lactam CTX-M-139
    G5DDS3 AEM44650.1 100.00 beta_lactam CTX-M-107
    R4HJN9 AFQ23953.1 100.00 beta_lactam SHV-147
    A0A0C5BN93 YP_008532725.1 100.00 beta_lactam KPC-2
    X2L731 CAA62957.1 100.00 beta_lactam CMY-2
    B0LQI6 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0N7HK16 AIS67613.1 100.00 beta_lactam CTX-M-157
    G5DDS9 AGW25368.1 100.00 beta_lactam CTX-M-142
    A0A0H4FL21 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    G8GMC0 BAM38093.1 100.00 beta_lactam IMP-34
    A0A127SAA2 BAF95849.1 100.00 beta_lactam SHV-111
    B0LS88 AAK13078.1 100.00 beta_lactam IMP-4
    B0LS87 AAK13078.1 100.00 beta_lactam IMP-4
    G3ESI4 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0P0GGR9 AGW25367.1 100.00 beta_lactam TEM-209
    X2KPW9 CAA62957.1 100.00 beta_lactam CMY-2
    A0ESF9 CAA76196.1 100.00 beta_lactam DHA-1
    A0ESF7 AFQ23961.1 100.00 beta_lactam SHV-155
    A0ESF4 AFX60298.1 100.00 beta_lactam CTX-M-134
    A0ESF3 ACP18863.1 100.00 beta_lactam CTX-M-88
    A0A140H9G6 ACV32633.1 100.00 beta_lactam SHV-123
    A0A0B5FZ76 AGC60012.1 100.00 beta_lactam OXA-244
    X2L7W9 CAA76196.1 100.00 beta_lactam DHA-1
    S5RWP1 AFX60298.1 100.00 beta_lactam CTX-M-134
    X2L7W4 AHM76771.1 100.00 beta_lactam ACT-17
    Q9R4I2 AFC75523.1 100.00 beta_lactam TEM-193
    Q1WBU0 CAI76927.1 100.00 beta_lactam SHV-12
    A0A0U2Q1H4 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0E4C3B8 AAK49460.1 100.00 beta_lactam OXA-35
    U3NAS3 ACV32635.1 100.00 beta_lactam SHV-125
    X2L748 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0U3S9M2 ABB72225.1 100.00 beta_lactam CTX-M-53
    A0A068B3Q2 ADI55020.1 100.00 beta_lactam PER-1
    A0A0U3S9M4 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0B5G2A9 AAL86924.1 100.00 beta_lactam CTX-M-22
    X5CCD9 AAT68658.1 100.00 beta_lactam CTX-M-10
    A0A023ZS83 AAP70012.1 100.00 beta_lactam OXA-48
    R4QIY1 AAK55534.1 100.00 beta_lactam CTX-M-19
    H9XQ07 ACF32746.1 100.00 beta_lactam TEM-163
    H9XQ04 CAA76793.1 100.00 beta_lactam TEM-20
    H9XQ05 AEM44654.1 100.00 beta_lactam CTX-M-109
    H9XQ02 ACV32633.1 100.00 beta_lactam SHV-123
    H9XQ03 ACP18864.1 100.00 beta_lactam TEM-169
    H9XQ00 ACV32633.1 100.00 beta_lactam SHV-123
    H9XQ01 ACV32633.1 100.00 beta_lactam SHV-123
    H9XQ09 ABY66120 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0N9RLE0 AFQ23954.1 100.00 beta_lactam SHV-148
    Q6PRU8 AAW82909 100.00 cephalosporin BL1_ec
    A0A0B5FPB4 AAL86924.1 100.00 beta_lactam CTX-M-22
    Q6PRU6 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    C8C096 AIA58911.1 100.00 beta_lactam OXA-1
    A0A0B5G0Y6 AAL86924.1 100.00 beta_lactam CTX-M-22
    A8EN07 CAG15145.1 100.00 beta_lactam OXA-59
    K0FCJ4 AAF27723.1 100.00 beta_lactam GES-1
    A0A127T7C5 BAF95849.1 100.00 beta_lactam SHV-111
    K0FCJ3 AAF27723.1 100.00 beta_lactam GES-1
    A0A154CVH0 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0K0QRP9 AAP70012.1 100.00 beta_lactam OXA-48
    G8EI99 AIT76115.1 100.00 beta_lactam MIR-13
    W0G6M6 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A140G9J3 CAA63262.1 100.00 beta_lactam CTX-M-1
    A0A140G9J4 AAF05311.2 100.00 beta_lactam CTX-M-9
    A0A140G9J7 CAD09800.1 100.00 beta_lactam TEM-1
    A0A140G9J6 CAI76927.1 100.00 beta_lactam SHV-12
    G1FI95 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A0M0MHW8 ZP_04228022 100.00 penicillin BL2a_III
    C5I508 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A157NY75 ABO42050 100.00 kanamycin---dibekacin---tobramycin---sisomicin---gentamicin Ant2Ia
    Q6UIU2 AAA24777 100.00 penicillin BL2a_pc
    F2YDU9 AAK17194.1 100.00 beta_lactam TEM-63
    A0A0G3FDP2 CAQ03505.1 100.00 beta_lactam SHV-100
    G0Z3A0 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0A0R181 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0H3W0W6 CAC16665 100.00 penicillin BL2a_pc
    A0A127T8C3 CAB42614.1 100.00 beta_lactam OXY-2-4
    A0A127T4F9 AET35493.1 100.00 beta_lactam OXA-347
    J7M7J2 AAL03985.1 100.00 beta_lactam TEM-71
    Q58GF5 ADI55020.1 100.00 beta_lactam PER-1
    Q58GF6 AEZ05108.1 100.00 beta_lactam GES-20
    D7SBW7 ABN09669.1 100.00 beta_lactam CTX-M-61
    A0A127SDC2 AEZ05108.1 100.00 beta_lactam GES-20
    A6N9J6 ADQ00385.1 100.00 beta_lactam DHA-7
    A0A088SUW3 ACV32635.1 100.00 beta_lactam SHV-125
    A0A127SF30 ACT97415.1 100.00 beta_lactam CblA-1
    A0A0X9U195 AGK40892.1 100.00 beta_lactam TEM-207
    B1PJP7 NP_052129.1 100.00 beta_lactam TEM-116
    U3N035 ACJ76644 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL3_imp
    S5RUC4 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A068HP14 CAC67290.1 100.00 beta_lactam TEM-95
    B8ZHZ1 CAI43422.1 100.00 beta_lactam OXY-6-1
    B8ZHZ3 CAI43425.1 100.00 beta_lactam OXY-6-4
    A0A0A0WDJ6 AGC60012.1 100.00 beta_lactam OXA-244
    D0V1N2 CAD09800.1 100.00 beta_lactam TEM-1
    D0V1N1 AAR99493.1 100.00 beta_lactam CTX-M-34
    B8ZHZ8 CAA88908.1 100.00 beta_lactam OXY-2-7
    A0A0P0G5D4 CEA29750.1 100.00 beta_lactam SHV-187
    A0A076V6B7 AEM44654.1 100.00 beta_lactam CTX-M-109
    H6UQI0 AAF72530.1 100.00 beta_lactam CTX-M-14
    H6UQI3 CAD70280.1 100.00 beta_lactam CTX-M-28
    X2JE62 AEM44654.1 100.00 beta_lactam CTX-M-109
    H6UQI5 ABN09669.1 100.00 beta_lactam CTX-M-61
    A0A0A7MDR2 YP_008532725.1 100.00 beta_lactam KPC-2
    H6UQI6 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0A0B5FPA8 AAP70012.1 100.00 beta_lactam OXA-48
    G0ZS39 NP_052129.1 100.00 beta_lactam TEM-116
    D1LFC4 AFX60298.1 100.00 beta_lactam CTX-M-134
    D1LFC5 AFX60298.1 100.00 beta_lactam CTX-M-134
    A0A141RLC5 ACJ05689.1 100.00 beta_lactam ACT-5
    X2L3X4 CAA76196.1 100.00 beta_lactam DHA-1
    A0A127SCF1 CAB42614.1 100.00 beta_lactam OXY-2-4
    A0A0M7N0G0 AAA23441 100.00 cephalosporin BL1_ec
    X2L736 AAL05857 100.00 cephalosporin BL1_ampc
    M9Z2Q0 AEM44651.1 100.00 beta_lactam CTX-M-108
    W8EDE9 AAF04388.1 100.00 beta_lactam CTX-M-8
    G0FGA7 ZP_03028124 100.00 kanamycin---dibekacin---tobramycin---sisomicin---gentamicin Ant2Ia
    X2L726 ADM21467.1 100.00 beta_lactam CMY-42
    A0A0X9RLK4 AGK40892.1 100.00 beta_lactam TEM-207
    B0LQH4 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    B0LQH7 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    B0LQH3 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    B0LQH2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0A0QZM8 AAL02127.1 100.00 beta_lactam CTX-M-15
    C5S4W2 CAA37052.1 100.00 beta_lactam ROB-1
    W8VGW8 CAA66729.1 100.00 beta_lactam SHV-11
    V9SDF6 ACJ76644 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL3_imp
    A5KRM7 ZP_02433069 100.00 chloramphenicol catA1
    G8GMB5 CAA11471.1 100.00 beta_lactam IMP-1
    S5ZYH8 CAB92324.1 100.00 beta_lactam TEM-68
    A0A0K0PXV2 AGC60012.1 100.00 beta_lactam OXA-244
    A0A0U3BLF7 AEM44654.1 100.00 beta_lactam CTX-M-109
    C8C167 CAC67290.1 100.00 beta_lactam TEM-95
    I6ZR36 AFQ23961.1 100.00 beta_lactam SHV-155
    A0A0D5XQH4 YP_001096379 100.00 streptomycin Aph33Ib
    A0A097SRU7 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0F6QQM8 AJO16044.1 100.00 beta_lactam TEM-214
    X2L7V1 AIT76113.1 100.00 beta_lactam MIR-9
    A0A0P0FJU5 CAA38429.1 100.00 beta_lactam TEM-2
    A0A0A0R218 CAD70280.1 100.00 beta_lactam CTX-M-28
    C9EF65 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0G3EIH5 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A0A0R177 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A0A0R173 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A127SAR9 NP_052129.1 100.00 beta_lactam TEM-116
    A0A145J891 AGU38146.1 100.00 beta_lactam ACT-23
    I6ZG67 CAI76927.1 100.00 beta_lactam SHV-12
    Q5MDA1 CAI76927.1 100.00 beta_lactam SHV-12
    A0A077CXG8 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    Q5MDA5 AAA87176.1 100.00 beta_lactam SHV-7
    X2KPV7 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0X9RET2 AGK40892.1 100.00 beta_lactam TEM-207
    A0A0E3W006 AIA58911.1 100.00 beta_lactam OXA-1
    K7X1Q4 AAG13410 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL2_kpc
    A0A0A0WBN9 BAM62793.1 100.00 beta_lactam IMP-42
    A0A068B0T0 AAK55534.1 100.00 beta_lactam CTX-M-19
    S4WTT5 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0N7FVM4 AAA87176.1 100.00 beta_lactam SHV-7
    E8LCT0 AAA23018.1 100.00 chloramphenicol cat
    G9F9U8 NP_052129.1 100.00 beta_lactam TEM-116
    A0A141RMV5 ACV32633.1 100.00 beta_lactam SHV-123
    A0A0N9NI12 AFY98865.1 100.00 beta_lactam CTX-M-139
    A0A0B4ZT09 ADQ00385.1 100.00 beta_lactam DHA-7
    A5HED8 AAG45720.1 100.00 beta_lactam OXA-10
    V9XY95 AFY98865.1 100.00 beta_lactam CTX-M-139
    B1A3K8 ZP_02433069 100.00 chloramphenicol catA1
    A6MFN4 NP_863283 100.00 penicillin BL2a_pc
    A0A077D028 CAI76927.1 100.00 beta_lactam SHV-12
    M1FVD1 CAD09800.1 100.00 beta_lactam TEM-1
    B0YKB7 NP_052129.1 100.00 beta_lactam TEM-116
    B0YKB8 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0A0R195 AAF72530.1 100.00 beta_lactam CTX-M-14
    A0A0K0VHH1 CAD09800.1 100.00 beta_lactam TEM-1
    E0XCT3 NP_863283 100.00 penicillin BL2a_pc
    A0A0D6A0J9 CAI76927.1 100.00 beta_lactam SHV-12
    A0A127T4G6 ADE59428.1 100.00 beta_lactam CARB-2
    A0A0G3Q585 AAT75225.1 100.00 beta_lactam SHV-30
    A0A0A0R6B0 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0A7QMS8 ZP_04114929 100.00 penicillin BL2a_III
    A0A0A0R6B9 ABB72225.1 100.00 beta_lactam CTX-M-53
    I6XPJ2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0F6QPS7 AGK82336.1 100.00 beta_lactam TEM-206
    A0A0H4IQA0 AIG51284.1 100.00 beta_lactam SHV-119
    A0A127STH5 YP_001144149 100.00 chloramphenicol catA13
    X2JHP6 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A127SB13 YP_002806354 100.00 cephalosporin BL1_ec
    K9N0F2 CAI76927.1 100.00 beta_lactam SHV-12
    A0A127SA83 AAA23441 100.00 cephalosporin BL1_ec
    X2JI23 CAD09800.1 100.00 beta_lactam TEM-1
    X2JI28 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0M4HEU4 AAP70012.1 100.00 beta_lactam OXA-48
    P0A6N2 NP_417798.1 100.00 elfamycin Escherichia
    S5RUB8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0U2ZDH1 BAM62793.1 100.00 beta_lactam IMP-42
    A0A088G689 CAA66659.1 100.00 beta_lactam TEM-42
    B5AR54 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0J0UE19 YP_001089504 100.00 bacitracin BacA
    A0A075M4I6 AEM44654.1 100.00 beta_lactam CTX-M-109
    G3JXZ0 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A0X9U929 AGK40892.1 100.00 beta_lactam TEM-207
    A0A0D5C9D9 ADO17948.1 100.00 beta_lactam CTX-M-98
    B1PJP9 NP_052129.1 100.00 beta_lactam TEM-116
    B1PJP8 NP_052129.1 100.00 beta_lactam TEM-116
    Q4TUB0 AAF72531.1 100.00 beta_lactam CTX-M-13
    H6UQJ5 CAD70280.1 100.00 beta_lactam CTX-M-28
    H6UQJ6 CAD70280.1 100.00 beta_lactam CTX-M-28
    D3J1E2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A141RPZ2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D1LFB3 AEM44650.1 100.00 beta_lactam CTX-M-107
    D1LFB6 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    W8ZWK1 CAS10898 100.00 bacitracin BacA
    D1LFB9 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    X2L3W1 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    V0SNI8 ACF32746.1 100.00 beta_lactam TEM-163
    A0A0K2FRZ6 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    J2Z214 YP_312012 100.00 bacitracin BacA
    D5L1K9 NP_052129.1 100.00 beta_lactam TEM-116
    W0IBE8 YP_001840873.1 100.00 beta_lactam OXA-58
    A0A0X9V180 AGK40892.1 100.00 beta_lactam TEM-207
    A0A0X9RLJ1 AGK40892.1 100.00 beta_lactam TEM-207
    W0IBE4 YP_001840873.1 100.00 beta_lactam OXA-58
    A0A0B4ZPC0 AIA58911.1 100.00 beta_lactam OXA-1
    A8HNT7 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A076PSH7 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0F6T731 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0F6T735 CAA63262.1 100.00 beta_lactam CTX-M-1
    A8HNT2 AAL02127.1 100.00 beta_lactam CTX-M-15
    V5LX70 ABB72225.1 100.00 beta_lactam CTX-M-53
    A0A0B5D5K3 YP_008532725.1 100.00 beta_lactam KPC-2
    Q6REX3 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A0M4FKR6 AAP70012.1 100.00 beta_lactam OXA-48
    S5RWN4 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A127T3S0 AAP69225.1 100.00 beta_lactam OXA-47
    A0A0A7QKR7 ZP_04115750 100.00 penicillin BL2a_1
    F1CGM9 ZP_02433069 100.00 chloramphenicol catA1
    A0A127SRC8 AFQ23961.1 100.00 beta_lactam SHV-155
    W0I7K4 YP_001840873.1 100.00 beta_lactam OXA-58
    A0A135X213 YP_002367214 100.00 penicillin BL2a_III
    F5RWR8 AJG06172.1 100.00 beta_lactam ACT-37
    A0A0A0R200 CAD89606.1 100.00 beta_lactam CTX-M-32
    A5YRZ5 CAC85660.1 100.00 beta_lactam TEM-93
    A0A0E1U443 CAF74882.1 100.00 beta_lactam OXA-57
    A0A0A0R206 CAA63262.1 100.00 beta_lactam CTX-M-1
    A0A0T9MH84 ZP_04637931 100.00 bacitracin BacA
    A6YRS7 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    K7Q9V3 CAD69003.1 100.00 beta_lactam imiH
    Q14F43 AAA23441 100.00 cephalosporin BL1_ec
    A0A0G3EII0 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A0H4FKW2 AAO61597.1 100.00 beta_lactam CTX-M-27
    U5L477 ZP_01172287 100.00 bacitracin BacA
    C5IDC8 ADA79630.1 100.00 beta_lactam TEM-171
    X2KPQ7 AAU95779.1 100.00 beta_lactam MIR-3
    G3JVG8 CAC85660.1 100.00 beta_lactam TEM-93
    V5LYI7 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0A0QZM1 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A0A0QZM5 CAD70280.1 100.00 beta_lactam CTX-M-28
    S5RFD5 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0U3B670 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A127SKB4 ACT97415.1 100.00 beta_lactam CblA-1
    A0A076Q255 CAG28417.1 100.00 beta_lactam CTX-M-96
    C3VJU9 AAA24777 100.00 penicillin BL2a_pc
    C3VJU7 CAC16665 100.00 penicillin BL2a_pc
    A0A0M4AYI5 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0F6T6C9 CAA63262.1 100.00 beta_lactam CTX-M-1
    A0A0F6T6C8 CAA63262.1 100.00 beta_lactam CTX-M-1
    Q6ZY68 AFN82060.1 100.00 beta_lactam SHV-33
    A0A0D5CAD7 ABY66120 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    Q6ZY66 CEA29750.1 100.00 beta_lactam SHV-187
    Q6ZY64 AFQ23961.1 100.00 beta_lactam SHV-155
    A0A0B5G0U3 AAP70012.1 100.00 beta_lactam OXA-48
    U4Q5P8 YP_001840873.1 100.00 beta_lactam OXA-58
    U2Y3I4 ZP_03555962 100.00 bacitracin BacA
    H4J5P3 CAS08424 100.00 macrolide MacB
    U5LPJ6 CAA11471.1 100.00 beta_lactam IMP-1
    W5SXK0 NP_052129.1 100.00 beta_lactam TEM-116
    A0A076PYK7 CAG28417.1 100.00 beta_lactam CTX-M-96
    A0A0N9JZY4 AAY43207.1 100.00 beta_lactam GES-9
    A0A0F7C900 AFY98865.1 100.00 beta_lactam CTX-M-139
    A8I4T6 AFQ23960.1 100.00 beta_lactam SHV-154
    A0A141RGJ3 YP_859822 100.00 cephalosporin BL1_ec
    A8I4T1 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A0D6HQ48 YP_859822 100.00 cephalosporin BL1_ec
    V5NAL7 AEM44654.1 100.00 beta_lactam CTX-M-109
    Z2ES40 BAC61820.1 100.00 beta_lactam CARB-22
    Q52639 AEG64812.1 100.00 beta_lactam TEM-185
    A0A0P0PSS9 ZP_04146665 100.00 penicillin BL2a_1
    X2JHQ2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    B6E4Z3 AAF72942.1 100.00 beta_lactam OXA-28
    X2JHQ6 AJO16044.1 100.00 beta_lactam TEM-214
    A0A127SB09 YP_001715362.1 100.00 beta_lactam VEB-1
    Q9JN58 CAA66659.1 100.00 beta_lactam TEM-42
    A0A127SIV4 ACP18863.1 100.00 beta_lactam CTX-M-88
    A0A127SFU3 CAG34105.1 100.00 beta_lactam TEM-129
    T2HGZ6 ZP_02433069 100.00 chloramphenicol catA1
    X2JI19 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0K0VJK2 AJP67510.1 100.00 beta_lactam GES-26
    U5LLC1 CAA11471.1 100.00 beta_lactam IMP-1
    A0A127SF66 AAL29433.1 100.00 beta_lactam TEM-81
    A0A127SF65 CAL85435.1 100.00 beta_lactam OXA-101
    A0A127SH46 AAR97270.1 100.00 beta_lactam GES-5
    U5LLC6 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A075M4V3 AEM44654.1 100.00 beta_lactam CTX-M-109
    V2SD25 AEG64812.1 100.00 beta_lactam TEM-185
    A0A136GGB2 CAG34043 100.00 ribostamycin---neomycin---paromomycin Aph3Vb
    G8HDC6 NP_052129.1 100.00 beta_lactam TEM-116
    C5IY20 AAR99493.1 100.00 beta_lactam CTX-M-34
    W5S484 CAA66659.1 100.00 beta_lactam TEM-42
    A0A0K0PYR7 AAP70012.1 100.00 beta_lactam OXA-48
    C7F8W9 AEM44654.1 100.00 beta_lactam CTX-M-109
    C7F8W8 AEM44654.1 100.00 beta_lactam CTX-M-109
    H6UQK3 AAK71471.1 100.00 beta_lactam CTX-M-17
    H6UQK0 AAF05311.2 100.00 beta_lactam CTX-M-9
    A0A127T6G9 ACV44455.1 100.00 beta_lactam OXY-2-9
    S5LNM4 AAK59385.1 100.00 beta_lactam IMP-9
    A0A076PVQ1 CAG28417.1 100.00 beta_lactam CTX-M-96
    X2L3V4 ADM21467.1 100.00 beta_lactam CMY-42
    A0A0D5C9I1 AAO61597.1 100.00 beta_lactam CTX-M-27
    G9J630 AEM44650.1 100.00 beta_lactam CTX-M-107
    A0A0F6P9I4 AAP70012.1 100.00 beta_lactam OXA-48
    E3VVM2 AGW25368.1 100.00 beta_lactam CTX-M-142
    B0LQJ0 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0P8HJV2 AGW25367.1 100.00 beta_lactam TEM-209
    A0A0K1LXN0 BAM62793.1 100.00 beta_lactam IMP-42
    A0A0A0R698 CAD89606.1 100.00 beta_lactam CTX-M-32
    M4QHJ9 NP_052129.1 100.00 beta_lactam TEM-116
    A0A095E5I8 YP_002092118 100.00 bacitracin BacA
    D7A3V9 BAF95849.1 100.00 beta_lactam SHV-111
    A0A076PSI4 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0S3B0H4 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0F6T726 CAD09800.1 100.00 beta_lactam TEM-1
    B1B3L5 BAE92844 100.00 penicillin BL2a_pc
    A0A060L046 BAD16611.1 100.00 beta_lactam CTX-M-36
    A0A0F7MZD0 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0A0A0WA09 AIL54055.1 100.00 beta_lactam CTX-M-115
    S5RWM3 CAA74912.2 100.00 beta_lactam TEM-17
    C9E0D8 AIA58911.1 100.00 beta_lactam OXA-1
    S5RWM7 CAI76927.1 100.00 beta_lactam SHV-12
    W1MR39 CAG30893 100.00 lividomycin---gentamicin---ribostamycin---kanamycin---neomycin---paromomycin Aph3Ib
    S5RAM0 CAI76927.1 100.00 beta_lactam SHV-12
    A0A127SAB0 AAA23441 100.00 cephalosporin BL1_ec
    A0A0X8GVX0 ABB72225.1 100.00 beta_lactam CTX-M-53
    A0A098CLR8 CAA66729.1 100.00 beta_lactam SHV-11
    M4Q6G4 NP_052129.1 100.00 beta_lactam TEM-116
    A0A068BFC9 CAA71321.1 100.00 beta_lactam CTX-M-3
    C5I4X0 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0N9JWR3 AAP70012.1 100.00 beta_lactam OXA-48
    Q5MDG6 AAL03985.1 100.00 beta_lactam TEM-71
    A0A0H4FKV6 CAG25427.1 100.00 beta_lactam TEM-135
    C8C0J7 ABG72340 100.00 cephalosporin BL1_ec
    P16873 ZP_03720331 100.00 penicillin pbp2b
    W5S441 CAA66659.1 100.00 beta_lactam TEM-42
    X2KPP7 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0F7MWG2 AAL86924.1 100.00 beta_lactam CTX-M-22
    S5RFE9 CAA66730.1 100.00 beta_lactam SHV-2A
    U6A4G3 CAB61645 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    Q53553 AAS89982.1 100.00 beta_lactam TEM-112
    Q6PRU7 CAD24670.1 100.00 beta_lactam TEM-30
    A0A029NPE3 ADB90239.1 100.00 beta_lactam TEM-176
    D2KB76 ACV32635.1 100.00 beta_lactam SHV-125
    A0A150DAP7 ZP_04072103 100.00 penicillin BL2a_III
    W8SN96 CAI76927.1 100.00 beta_lactam SHV-12
    A0A097DET4 CAA71321.1 100.00 beta_lactam CTX-M-3
    T1Z0U3 AFY98865.1 100.00 beta_lactam CTX-M-139
    S5U6S7 NP_052129.1 100.00 beta_lactam TEM-116
    A0A127SF60 CAA29443.1 100.00 beta_lactam OXA-2
    A0A0H4IR39 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0B5FZ28 AGC60012.1 100.00 beta_lactam OXA-244
    A0A0F6TL49 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A141RGI1 CAI43413.1 100.00 beta_lactam OXY-1-2
    A0A088FNH9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    C5S3S4 CAA37052.1 100.00 beta_lactam ROB-1
    A0A0J1M759 AEG64812.1 100.00 beta_lactam TEM-185
    Q6Q4T6 AAR89359.1 100.00 beta_lactam TEM-128
    Q9KHC0 CAA74912.2 100.00 beta_lactam TEM-17
    G3KE67 CAD09800.1 100.00 beta_lactam TEM-1
    E0XCV6 YP_001569085.1 100.00 beta_lactam PC1
    E0XCV8 CAC16665 100.00 penicillin BL2a_pc
    A0A141RIZ6 AHL39322.1 100.00 beta_lactam CMY-85
    V5NAM5 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A127SS46 ACB20240 100.00 chloramphenicol catA2
    C0L085 ADE59428.1 100.00 beta_lactam CARB-2
    A0A0U2W4P1 ABP04245.1 100.00 beta_lactam CTX-M-62
    F1DI17 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A0R7FFW3 ACF32746.1 100.00 beta_lactam TEM-163
    A0A141RN86 AAW82909 100.00 cephalosporin BL1_ec
    B5TTG3 YP_001096379 100.00 streptomycin Aph33Ib
    A0T1X9 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0G3V258 AAA25053.1 100.00 beta_lactam TEM-12
    A0A0F6R914 ACS32294.1 100.00 beta_lactam CTX-M-75
    J9XUS1 ABN69105.1 100.00 beta_lactam CTX-M-65
    A0A141RML6 AAS89982.1 100.00 beta_lactam TEM-112
    A0A0F6J6Q0 ZP_04103077 100.00 penicillin BL2a_1
    A0A0U1I9I1 ZP_04640456 100.00 bacitracin BacA
    X2JI04 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0H4LWD9 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A127SHA4 CAI46978 100.00 penicillin---carbenicillin BL2c_pse3
    I6ZM27 AFQ94051.1 100.00 beta_lactam CTX-M-132
    A5A5H3 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    A5A5H1 YP_001840873.1 100.00 beta_lactam OXA-58
    A0A142EAP3 YP_001144148 100.00 streptomycin---spectinomycin Ant3Ia
    G3GCC3 YP_008532725.1 100.00 beta_lactam KPC-2
    G3GCC7 YP_008532725.1 100.00 beta_lactam KPC-2
    G3GCC6 YP_008532725.1 100.00 beta_lactam KPC-2
    G3GCC5 YP_008532725.1 100.00 beta_lactam KPC-2
    Q2MJ77 AJP67510.1 100.00 beta_lactam GES-26
    B2CD72 BAM28879.1 100.00 beta_lactam SHV-167
    A0A0D5C9H5 AEM44654.1 100.00 beta_lactam CTX-M-109
    X2JNP6 ACF32746.1 100.00 beta_lactam TEM-163
    A0A0A0WK39 ZP_04263080 100.00 penicillin BL2a_1
    A0A127SDV3 AAK15582.1 100.00 beta_lactam OXA-34
    W0I7E7 YP_001840873.1 100.00 beta_lactam OXA-58
    Q2FC46 NP_863283 100.00 penicillin BL2a_pc
    Q2FC42 NP_863283 100.00 penicillin BL2a_pc
    A0A076PSJ1 ACV32635.1 100.00 beta_lactam SHV-125
    A0A0K0PXK2 AAP70012.1 100.00 beta_lactam OXA-48
    A0A076VEF4 AAZ95872.1 100.00 beta_lactam CTX-M-2
    Q2FC48 NP_863283 100.00 penicillin BL2a_pc
    A0A0K0PXK5 AAP70012.1 100.00 beta_lactam OXA-48
    G8EX95 YP_855286 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL3_cphA
    A0A0U3TBQ9 CAA30578.1 100.00 aminoglycoside APH(3')-VIa
    A0A141RDQ3 CAI43425.1 100.00 beta_lactam OXY-6-4
    A0A127T5K4 CAB42614.1 100.00 beta_lactam OXY-2-4
    X2L5I3 AFA51701.1 100.00 beta_lactam CTX-M-123
    Q8RL89 ACQ82813.1 100.00 beta_lactam PDC-8
    X2L5I8 AIS67613.1 100.00 beta_lactam CTX-M-157
    A0A0S2LLL0 AAP70012.1 100.00 beta_lactam OXA-48
    M4Q6F9 NP_052129.1 100.00 beta_lactam TEM-116
    E0XN37 AAD37403.1 100.00 beta_lactam TLA-1
    A0A0B5FZ32 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0B5FZ35 AAP70012.1 100.00 beta_lactam OXA-48
    Q4GY26 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    H9NAS8 ACV32635.1 100.00 beta_lactam SHV-125
    H6UQK2 AAV70602.1 100.00 beta_lactam CTX-M-38
    S5RAL1 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0F7NK96 ACP18863.1 100.00 beta_lactam CTX-M-88
    X2KW07 CAA76196.1 100.00 beta_lactam DHA-1
    X2KW05 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    X2KW01 CAA76196.1 100.00 beta_lactam DHA-1
    H9NAN9 AEM44651.1 100.00 beta_lactam CTX-M-108
    A0A0U5EL28 CAC67290.1 100.00 beta_lactam TEM-95
    A0A0G3FB78 CAI76927.1 100.00 beta_lactam SHV-12
    C8C0K5 ABG72340 100.00 cephalosporin BL1_ec
    A0A141REZ3 AAO53445.1 100.00 beta_lactam SHV-46
    A0A127SFF9 YP_001144149 100.00 chloramphenicol catA13
    A0A088SAV5 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A141REZ9 CEA29751.1 100.00 beta_lactam SHV-188
    A0A023ZSW7 AAP70012.1 100.00 beta_lactam OXA-48
    X2KPS3 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0J0GHG4 AAC45086.2 100.00 beta_lactam ACT-1
    R9URP6 AAL02127.1 100.00 beta_lactam CTX-M-15
    U3NF58 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0M5II90 AEM44654.1 100.00 beta_lactam CTX-M-109
    Q849R6 AAQ75378 100.00 monobactam---penicillin BL2be_oxy1
    Q849R7 CAA82916.1 100.00 beta_lactam OXY-1-1
    Q849R4 AAQ75378 100.00 monobactam---penicillin BL2be_oxy1
    Q849R5 CAB42615.1 100.00 beta_lactam OXY-1-6
    A0A097DES2 AIL54055.1 100.00 beta_lactam CTX-M-115
    Q849R3 CAA82916.1 100.00 beta_lactam OXY-1-1
    A0A097DES0 AAP70012.1 100.00 beta_lactam OXA-48
    Q849R8 AAL78278.2 100.00 beta_lactam OXY-2-6
    A0A0K1P829 AJO16041.1 100.00 beta_lactam SHV-180
    A0A0B5G0S8 AAP70012.1 100.00 beta_lactam OXA-48
    A0A076PYI9 AEM44654.1 100.00 beta_lactam CTX-M-109
    I3QI22 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A0X9U5L8 AAY85632.1 100.00 beta_lactam TEM-143
    A0A023WGZ6 CAD70280.1 100.00 beta_lactam CTX-M-28
    F1DQF7 AAR32651.1 100.00 beta_lactam OXA-56
    E9NGZ2 CAI76927.1 100.00 beta_lactam SHV-12
    A0A0B5E4A1 NP_052129.1 100.00 beta_lactam TEM-116
    A0A127SSG7 CAI76927.1 100.00 beta_lactam SHV-12
    Q9AQH2 AAK17194.1 100.00 beta_lactam TEM-63
    A0A120HYV4 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    W8J9L5 AGR55864.1 100.00 beta_lactam OXA-320
    A6B9Y3 AGZ20205.1 100.00 beta_lactam TEM-205
    D2Z1S2 AAK71471.1 100.00 beta_lactam CTX-M-17
    D2Z1S3 AAO61597.1 100.00 beta_lactam CTX-M-27
    B5LY46 CAD89606.1 100.00 beta_lactam CTX-M-32
    B5LY47 AAO61597.1 100.00 beta_lactam CTX-M-27
    W0NTG4 YP_001840873.1 100.00 beta_lactam OXA-58
    H4K0L1 CAS08424 100.00 macrolide MacB
    A0A0U1YUD5 AAP70012.1 100.00 beta_lactam OXA-48
    A0A073VGB3 AAS89982.1 100.00 beta_lactam TEM-112
    A0A066PEV6 AAT97115 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A0K1YWJ5 AFY98865.1 100.00 beta_lactam CTX-M-139
    A0A0X9SH32 AGK82336.1 100.00 beta_lactam TEM-206
    S4WGH4 ACV92002.1 100.00 beta_lactam CTX-M-71
    A0A0U4C4J6 NP_052129.1 100.00 beta_lactam TEM-116
    R9XVF2 AIH07017.1 100.00 beta_lactam KPC-19
    R9XVF4 AAZ95872.1 100.00 beta_lactam CTX-M-2
    A0A0A1ELL2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D7SBX0 AIA58911.1 100.00 beta_lactam OXA-1
    D7SBX1 AET05998.1 100.00 beta_lactam OXA-4
    A0A0D5CSF3 CAA63262.1 100.00 beta_lactam CTX-M-1
    Q4TUA9 AAF72530.1 100.00 beta_lactam CTX-M-14
    Q4TUA8 AAN38836.1 100.00 beta_lactam CTX-M-24
    A0A141RCK8 ADP02979.1 100.00 beta_lactam CMY-48
    B2CD61 AAG23871.1 100.00 beta_lactam OXA-33
    A8HNT5 AFY98865.1 100.00 beta_lactam CTX-M-139
    B2CD65 CAA66659.1 100.00 beta_lactam TEM-42
    B2CD64 CAA66659.1 100.00 beta_lactam TEM-42
    B2CD68 CAA66659.1 100.00 beta_lactam TEM-42
    X2JNQ1 CAD09800.1 100.00 beta_lactam TEM-1
    X2JNQ7 CAD09800.1 100.00 beta_lactam TEM-1
    H6UQG9 CAG34237.1 100.00 beta_lactam OXA-1
    A0A0H4SZ16 CAC85157 100.00 cephalosporin BL1_ampc
    B6VPY7 BAD16611.1 100.00 beta_lactam CTX-M-36
    S5MBL8 AAK59385.1 100.00 beta_lactam IMP-9
    A0A0H4FVD9 AAL02127.1 100.00 beta_lactam CTX-M-15
    Q2FC54 AAA24777 100.00 penicillin BL2a_pc
    Q2FC52 AAA24777 100.00 penicillin BL2a_pc
    A0A127T3B0 AAN41430 100.00 cloxacillin---penicillin BL2d_oxa1
    T1SPE6 CAG28417.1 100.00 beta_lactam CTX-M-96
    A0A0P0M0H1 BAM62793.1 100.00 beta_lactam IMP-42
    A0A023ZSF9 AGC60012.1 100.00 beta_lactam OXA-244
    I7AKP2 AIH07017.1 100.00 beta_lactam KPC-19
    C5J4Z0 AAA22904.1 100.00 beta_lactam CcrA
    I7AKP5 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0H4ITP5 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0U5BJT0 CAG25427.1 100.00 beta_lactam TEM-135
    A6BBS7 AGZ20205.1 100.00 beta_lactam TEM-205
    C9EF68 CAD70280.1 100.00 beta_lactam CTX-M-28
    W8EEE7 AAN73274 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A097SRQ9 NP_052129.1 100.00 beta_lactam TEM-116
    X2L7Z5 CAA76196.1 100.00 beta_lactam DHA-1
    A0A097SRQ0 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0A7M998 ACB71165.1 100.00 beta_lactam KPC-6
    A0A0P0HX21 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0H3QF17 ZP_01974653 100.00 streptomycin---spectinomycin Ant3Ia
    A0A076V4Z2 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A076V4Z5 CAG28417.1 100.00 beta_lactam CTX-M-96
    A0A0A1DZL3 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    A0A077L7Q2 AAO61597.1 100.00 beta_lactam CTX-M-27
    X2KW19 CAA76196.1 100.00 beta_lactam DHA-1
    A0A098CLP7 AFQ23961.1 100.00 beta_lactam SHV-155
    T1WFF2 AEM44654.1 100.00 beta_lactam CTX-M-109
    X2KW14 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0U2V4T1 NP_052129.1 100.00 beta_lactam TEM-116
    D1LDK9 AAF05311.2 100.00 beta_lactam CTX-M-9
    A0A0A0WAV9 CAA71321.1 100.00 beta_lactam CTX-M-3
    A0A0U2KCI2 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0M4N734 CAI76927.1 100.00 beta_lactam SHV-12
    A0A0S3B097 AAF04388.1 100.00 beta_lactam CTX-M-8
    A0A0H4FNT6 AFN82060.1 100.00 beta_lactam SHV-33
    A0A0H4FQ48 ABN69105.1 100.00 beta_lactam CTX-M-65
    X2KPR8 CAA75402.1 100.00 beta_lactam CMY-4
    A0A127T3T5 CAA29443.1 100.00 beta_lactam OXA-2
    X2KPR1 AHM76771.1 100.00 beta_lactam ACT-17
    A0A0M5IDZ4 AAC41391.1 100.00 aminoglycoside AAC(6')-Ih
    A0A0X9SCJ7 CAA76796.1 100.00 beta_lactam TEM-29
    A0A127SIT2 AIT76098.1 100.00 beta_lactam CMY-119
    A0A088FSW7 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0H4FLF6 ABI34705.1 100.00 beta_lactam CTX-M-55
    E5KCK4 ADI55020.1 100.00 beta_lactam PER-1
    Q6ZY52 AAK69830.1 100.00 beta_lactam LEN-2
    C5I542 AAL86924.1 100.00 beta_lactam CTX-M-22
    C5I544 AAK71471.1 100.00 beta_lactam CTX-M-17
    C5I549 AAF05311.2 100.00 beta_lactam CTX-M-9
    Q849S0 AAQ75378 100.00 monobactam---penicillin BL2be_oxy1
    A0A0H4FV83 AFN82060.1 100.00 beta_lactam SHV-33
    Q849S9 AAL78278.2 100.00 beta_lactam OXY-2-6
    Q849S8 AAL78278.2 100.00 beta_lactam OXY-2-6
    A0A088FNJ8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    C0LE18 ABN48311.1 100.00 beta_lactam CTX-M-56
    A0A023WGE8 AGW25368.1 100.00 beta_lactam CTX-M-142
    A0A0K0QSK4 AAP70012.1 100.00 beta_lactam OXA-48
    D1MIA7 NP_052129.1 100.00 beta_lactam TEM-116
    S5R7S1 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    S5R7S6 CAD89606.1 100.00 beta_lactam CTX-M-32
    V5NHC7 AGR55864.1 100.00 beta_lactam OXA-320
    A0A0C2DUQ5 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    A0A0B5FRN8 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0S2MT55 CAD70280.1 100.00 beta_lactam CTX-M-28
    G0Z389 CAO98721.1 100.00 beta_lactam TEM-15
    U3PRJ6 YP_008532725.1 100.00 beta_lactam KPC-2
    V5RJP5 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0T8JTR7 ZP_04192734 100.00 penicillin BL2a_1
    Q5MD97 AAT97115 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    X2JG39 AEM44654.1 100.00 beta_lactam CTX-M-109
    R9WWA8 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A089MVM8 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    X2L804 CAA76196.1 100.00 beta_lactam DHA-1
    I6ZGA9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    F6KHL8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    R9URM7 AEL88240.1 100.00 beta_lactam TEM-190
    A0A0K0VJH5 AEZ05107.1 100.00 beta_lactam GES-19
    X2GB66 YP_001840873.1 100.00 beta_lactam OXA-58
    M1HNY5 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0E1SGW2 CAG15145.1 100.00 beta_lactam OXA-59
    A0A127SCU0 CAA29443.1 100.00 beta_lactam OXA-2
    B5B789 AAL02127.1 100.00 beta_lactam CTX-M-15
    W0IC35 YP_001840873.1 100.00 beta_lactam OXA-58
    A0A0S1RU79 ABX38721.1 100.00 beta_lactam OXA-114a
    X2L6C3 AEM44650.1 100.00 beta_lactam CTX-M-107
    V5NAA5 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A140IK96 AJO16044.1 100.00 beta_lactam TEM-214
    A0A140IK97 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    V5NAA1 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A140IK98 AEM44651.1 100.00 beta_lactam CTX-M-108
    A8HNS3 CAD70280.1 100.00 beta_lactam CTX-M-28
    A8HNS0 CAD70280.1 100.00 beta_lactam CTX-M-28
    A8HNS8 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0N9ZUP0 AFY98865.1 100.00 beta_lactam CTX-M-139
    A0A0J1QHQ8 AGK82336.1 100.00 beta_lactam TEM-206
    A0A155RV88 AHL39338.1 100.00 beta_lactam ACT-25
    A0A141RPT2 CAI43423.1 100.00 beta_lactam OXY-6-2
    A0A0F7RBX2 AAR20595.1 100.00 beta_lactam BLA1
    U5LMD0 YP_008532725.1 100.00 beta_lactam KPC-2
    B8ZHZ4 CAI43413.1 100.00 beta_lactam OXY-1-2
    Q6QIV0 CAD09800.1 100.00 beta_lactam TEM-1
    X2JNL8 AEM44654.1 100.00 beta_lactam CTX-M-109
    B8ZHZ6 ACV44456.1 100.00 beta_lactam OXY-2-10
    W6GIQ5 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0P0PT51 ZP_04145743 100.00 penicillin BL2a_III
    A0A0A0R793 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0N8MNQ9 ABM54870.1 100.00 beta_lactam TEM-160
    K9US24 ACM91559.1 100.00 beta_lactam KPC-9
    A0A0E3ZB52 ABY91240.1 100.00 beta_lactam KPC-5
    Q6IWJ0 ZP_02433069 100.00 chloramphenicol catA1
    A0A075M8X1 CAG28417.1 100.00 beta_lactam CTX-M-96
    F1C6A2 CAG25427.1 100.00 beta_lactam TEM-135
    A0A0E3KBA7 BAE71359 100.00 penicillin---carbenicillin BL2c_pse1
    A0A0D4D8H0 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A077JMN5 ACS32294.1 100.00 beta_lactam CTX-M-75
    X2L7Y1 AEI52842.1 100.00 beta_lactam CMY-65
    A0A081M7B9 CAA66729.1 100.00 beta_lactam SHV-11
    X2L7Y6 AIT76111.1 100.00 beta_lactam MIR-10
    F5CEB7 CAG28417.1 100.00 beta_lactam CTX-M-96
    H6WND4 ACR66837 100.00 cloxacillin---penicillin BL2d_oxa10
    A0A076V4Y0 CAA63262.1 100.00 beta_lactam CTX-M-1
    Q5D4P6 CAA11471.1 100.00 beta_lactam IMP-1
    A0A109YG28 ACF32746.1 100.00 beta_lactam TEM-163
    W8E8N7 CAO98721.1 100.00 beta_lactam TEM-15
    A0A141RBC1 BAF95849.1 100.00 beta_lactam SHV-111
    A3FEW4 CAA71321.1 100.00 beta_lactam CTX-M-3
    A3FEW5 AFY98865.1 100.00 beta_lactam CTX-M-139
    K9M7C1 CAA66659.1 100.00 beta_lactam TEM-42
    A0A0Q0CKX8 EEB58471 100.00 bacitracin BacA
    K9M2H9 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    Q9X5K9 CAA73933.1 100.00 beta_lactam TEM-52
    A0A0F6Y5C7 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0H4FNU6 ADY02555.1 100.00 beta_lactam CTX-M-104
    A0A0H4FNU1 AGW25368.1 100.00 beta_lactam CTX-M-142
    A0A0M5L4Q3 ACV32635.1 100.00 beta_lactam SHV-125
    K0F407 AAT97115 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A0Y0MK15 ABS58643 100.00 cephalosporin BL1_ec
    A0A0N9NXT5 CAI76927.1 100.00 beta_lactam SHV-12
    A0A128DXD0 ADK34116.1 100.00 beta_lactam OXA-164
    Q4PQY9 ABF70513.1 100.00 beta_lactam IMP-30
    A0A0G3FHC6 ADO17948.1 100.00 beta_lactam CTX-M-98
    A0A0H4FLG2 ABN69105.1 100.00 beta_lactam CTX-M-65
    A0A0D5CAH8 AAO61597.1 100.00 beta_lactam CTX-M-27
    A0A109ZX36 ABS90365.1 100.00 beta_lactam CTX-M-67
    K7XF77 AAG13410 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL2_kpc
    A0A0D5CAH5 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0E3T6N7 ACP18863.1 100.00 beta_lactam CTX-M-88
    I6QLD9 AAF05311.2 100.00 beta_lactam CTX-M-9
    A0A0B5ECT7 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0N9ZZX6 AFA51701.1 100.00 beta_lactam CTX-M-123
    I7AKM8 CAA76196.1 100.00 beta_lactam DHA-1
    L7SWR9 ACQ82810.1 100.00 beta_lactam PDC-5
    V5N9T7 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0S2PJW1 CAI76927.1 100.00 beta_lactam SHV-12
    Q2V0R5 ZP_01974653 100.00 streptomycin---spectinomycin Ant3Ia
    A0A0D8VML0 ZP_03029420 100.00 macrolide MacB
    A0A0M3R833 AGC60012.1 100.00 beta_lactam OXA-244
    D1LFC1 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    V5Q693 AJP67510.1 100.00 beta_lactam GES-26
    D1LFC3 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    T1Y7H4 BAF77563 100.00 bacitracin BacA
    G0Z391 CAO98721.1 100.00 beta_lactam TEM-15
    G0Z393 CEA29750.1 100.00 beta_lactam SHV-187
    G0Z392 ACV32637.1 100.00 beta_lactam SHV-127
    G0Z394 ACR66323.1 100.00 beta_lactam SHV-1
    G0Z397 CAI10727.2 100.00 beta_lactam SHV-55
    G0Z396 AAN39364.1 100.00 beta_lactam SHV-45
    A0A0N7FYW3 AAR99493.1 100.00 beta_lactam CTX-M-34
    D1LFC9 BAF63422.1 100.00 beta_lactam CTX-M-64
    Q849T0 CAA88908.1 100.00 beta_lactam OXY-2-7
    A0A127SFG3 AGW25367.1 100.00 beta_lactam TEM-209
    A0A127SFG6 YP_001144149 100.00 chloramphenicol catA13
    A0A077D5K4 ABA00479.1 100.00 beta_lactam IMI-2
    A0A0P9J8U8 ZP_04589788 100.00 bacitracin BacA
    U1U5A0 CAC16665 100.00 penicillin BL2a_pc
    D4NZD2 AIS67611.1 100.00 beta_lactam CTX-M-155
    A0A127SA56 CAJ47133.2 100.00 beta_lactam SHV-78
    A0A0R7FDJ3 CAD09800.1 100.00 beta_lactam TEM-1
    B8YE30 CAB92324.1 100.00 beta_lactam TEM-68
    I6UZ34 AJO16039.1 100.00 beta_lactam IMP-47
    A0A141RJP4 AAF86692 100.00 cephalosporin---penicillin BL1_acc
    K9M478 AAG45720.1 100.00 beta_lactam OXA-10
    A0A0P0AJZ1 AIS67613.1 100.00 beta_lactam CTX-M-157
    Q5Q116 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    X5NYF0 ZP_04433866 100.00 bacitracin BacA
    C5IY65 AAL02127.1 100.00 beta_lactam CTX-M-15
    K7XAF0 CAB92324.1 100.00 beta_lactam TEM-68
    A0A0M2KFM7 ABO64442.1 100.00 beta_lactam TEM-162
    R9UUF5 ACV32635.1 100.00 beta_lactam SHV-125
    I3WB62 ABF67771 100.00 streptomycin---spectinomycin Ant3Ia
    A8FGS0 YP_001487997 100.00 bacitracin BacA
    A7Y412 WP_004746565.1 100.00 beta_lactam ADC-2
    A0A089GHT1 ADI55020.1 100.00 beta_lactam PER-1
    B2CD49 CAA66659.1 100.00 beta_lactam TEM-42
    A8HNR2 CAD70280.1 100.00 beta_lactam CTX-M-28
    A8HNR5 ACP18863.1 100.00 beta_lactam CTX-M-88
    A8HNR7 CAD70280.1 100.00 beta_lactam CTX-M-28
    B2CD46 CAA66659.1 100.00 beta_lactam TEM-42
    A7U8M8 CAA79968.1 100.00 beta_lactam PER-1
    A0A0B5FRU9 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A127SQM0 ABN69105.1 100.00 beta_lactam CTX-M-65
    X2L741 CAA76196.1 100.00 beta_lactam DHA-1
    X2JE57 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A0E3ZNN0 ZP_02433069 100.00 chloramphenicol catA1
    B0LQH9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    B6VP05 ABN51007.1 100.00 beta_lactam CMY-35
    A0A127T5H5 ACB20240 100.00 chloramphenicol catA2
    Q6GYC6 BAM34463.1 100.00 beta_lactam ACT-16
    Q0Q297 CAA63262.1 100.00 beta_lactam CTX-M-1
    Q0Q299 CAA63262.1 100.00 beta_lactam CTX-M-1
    Q0Q298 CAA62957.1 100.00 beta_lactam CMY-2
    T1SGU7 AIL54055.1 100.00 beta_lactam CTX-M-115
    A0A097SRS3 NP_052129.1 100.00 beta_lactam TEM-116
    A0A097SRS5 NP_052129.1 100.00 beta_lactam TEM-116
    A0A097SRS6 NP_052129.1 100.00 beta_lactam TEM-116
    A0A097SRR6 NP_052129.1 100.00 beta_lactam TEM-116
    X5CTH9 AGC60012.1 100.00 beta_lactam OXA-244
    E5LLI3 ADY02545.1 100.00 beta_lactam CTX-M-101
    A0A0E0VZD6 AAR20595.1 100.00 beta_lactam BLA1
    M4Q6K8 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0A7HIW2 ADB90239.1 100.00 beta_lactam TEM-176
    J7H5N2 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A0A1AKJ9 ABA54978 100.00 cloxacillin---penicillin BL2d_oxa9
    A0A141RKA3 AAF86692 100.00 cephalosporin---penicillin BL1_acc
    X2JNM9 AEM44654.1 100.00 beta_lactam CTX-M-109
    V0YC62 AAG23871.1 100.00 beta_lactam OXA-33
    B5AR55 AAN04882.1 100.00 beta_lactam SHV-40
    F1C695 CAA08843 100.00 cloxacillin---penicillin BL2d_oxa1
    I6ZFV8 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A059ULV6 CAA66659.1 100.00 beta_lactam TEM-42
    F1C691 AEM44650.1 100.00 beta_lactam CTX-M-107
    F1C690 ACV32635.1 100.00 beta_lactam SHV-125
    A0A0H4LT62 YP_008532725.1 100.00 beta_lactam KPC-2
    F1C698 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0J1KPJ8 AFK73434.1 100.00 beta_lactam CMY-75
    A0A0A0WBT5 AIL54055.1 100.00 beta_lactam CTX-M-115
    K7WC36 CAB92324.1 100.00 beta_lactam TEM-68
    A0A127T3V7 BAE71359 100.00 penicillin---carbenicillin BL2c_pse1
    X2LF25 AIT76104.1 100.00 beta_lactam MIR-15
    A0A076V7F7 ADE59428.1 100.00 beta_lactam CARB-2
    W8QL15 AAL03985.1 100.00 beta_lactam TEM-71
    D1LU79 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    I6XG69 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    M4PZF0 BAM62793.1 100.00 beta_lactam IMP-42
    X5CFE8 CAD70280.1 100.00 beta_lactam CTX-M-28
    S4WLS7 ACP18863.1 100.00 beta_lactam CTX-M-88
    T1WEK3 CAG28417.1 100.00 beta_lactam CTX-M-96
    A0A068JFB7 CAI76927.1 100.00 beta_lactam SHV-12
    Q8VQ00 AAL99990.1 100.00 beta_lactam CTX-M-23
    L0HT31 AAL02127.1 100.00 beta_lactam CTX-M-15
    I7AKN7 CAA76196.1 100.00 beta_lactam DHA-1
    Q5MB16 AAG45720.1 100.00 beta_lactam OXA-10
    A0A0N9K6X8 AJP67510.1 100.00 beta_lactam GES-26
    V5N9U2 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0B5EAU8 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0S2PJV3 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0B5EAU4 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0B5EAU0 NP_052129.1 100.00 beta_lactam TEM-116
    U3N8F4 CAA71321.1 100.00 beta_lactam CTX-M-3
    A0A0S3FMD5 AAA87176.1 100.00 beta_lactam SHV-7
    A0A0U3SX12 ABN51006.1 100.00 beta_lactam CMY-34
    S5LSG4 CAD89606.1 100.00 beta_lactam CTX-M-32
    R9RLU1 ACF32746.1 100.00 beta_lactam TEM-163
    A0A127SS40 CAA62957.1 100.00 beta_lactam CMY-2
    A6MFN5 NP_765163 100.00 penicillin BL2a_pc
    A6MFN6 NP_863283 100.00 penicillin BL2a_pc
    A6MFN1 CAC16665 100.00 penicillin BL2a_pc
    A6MFN2 AAA24777 100.00 penicillin BL2a_pc
    A6MFN3 NP_863283 100.00 penicillin BL2a_pc
    S5R7Q9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    S5R7Q4 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0F6J1G5 ZP_04102233 100.00 penicillin BL2a_III
    G8IIS9 ABI34705.1 100.00 beta_lactam CTX-M-55
    A0A0A7HIV7 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0U5C6U4 YP_859822 100.00 cephalosporin BL1_ec
    A0A127SIA7 ACT97415.1 100.00 beta_lactam CblA-1
    A0A141R9A4 AJO16044.1 100.00 beta_lactam TEM-214
    G9G8D3 AAG13410 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL2_kpc
    D2KHP3 AAK12087.1 100.00 beta_lactam IMP-7
    E7DK53 ACA34343 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL2_kpc
    A0A127SA64 ACV32636.1 100.00 beta_lactam SHV-126
    F1CKJ8 AAT68658.1 100.00 beta_lactam CTX-M-10
    A0A024QIW3 AEG64812.1 100.00 beta_lactam TEM-185
    S6HVN0 AAF86642.1 100.00 glycopeptide vanSC
    A0A140DR56 AAO85882.1 100.00 beta_lactam TEM-120
    A0A0U1YUG2 AFU91598.1 100.00 beta_lactam OXA-204
    A0A0G3FAD5 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0K2FSG6 CEA29750.1 100.00 beta_lactam SHV-187
    Q8RNA0 AAF05311.2 100.00 beta_lactam CTX-M-9
    A0A141RLT2 AHL39322.1 100.00 beta_lactam CMY-85
    E1APJ2 CAA63262.1 100.00 beta_lactam CTX-M-1
    Q8RSQ8 NP_862669.1 100.00 aminoglycoside APH(3'')-Ib
    A0A127SWP1 YP_002806354 100.00 cephalosporin BL1_ec
    A0A088FSW1 ABO64442.1 100.00 beta_lactam TEM-162
    E2ITW2 AAO61597.1 100.00 beta_lactam CTX-M-27
    G8A3U9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0S1RS54 ABX38721.1 100.00 beta_lactam OXA-114a
    P14510 AAL82588.1 100.00 aminoglycoside AAC(3)-Ib/AAC(6')-Ib''
    E2E9M1 AAA98406.1 100.00 beta_lactam OXA-9
    A6MFP2 NP_863283 100.00 penicillin BL2a_pc
    A6MFP3 NP_863283 100.00 penicillin BL2a_pc
    G0ZS46 NP_052129.1 100.00 beta_lactam TEM-116
    G0ZS44 NP_052129.1 100.00 beta_lactam TEM-116
    G0ZS45 NP_052129.1 100.00 beta_lactam TEM-116
    G0ZS42 NP_052129.1 100.00 beta_lactam TEM-116
    A8HNQ9 ACP18863.1 100.00 beta_lactam CTX-M-88
    G0ZS40 NP_052129.1 100.00 beta_lactam TEM-116
    E2GDC5 AAK29174.1 100.00 beta_lactam VEB-2
    A0A142BM49 ABS58643 100.00 cephalosporin BL1_ec
    A0A088FNH5 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0D5CA73 ABY66120 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    E3Q1V4 ACP18863.1 100.00 beta_lactam CTX-M-88
    D3JYV7 ACV32635.1 100.00 beta_lactam SHV-125
    A0A090NHY8 Q32DT3 100.00 peptide---polymyxin arnA
    D3JYV9 ACV32635.1 100.00 beta_lactam SHV-125
    A0A0D5J0Q1 YP_002099037 100.00 bacitracin BacA
    S5TGS9 AIH07017.1 100.00 beta_lactam KPC-19
    A0A0B5G2B3 CAD70280.1 100.00 beta_lactam CTX-M-28
    A9QNW7 ZP_03028124 100.00 kanamycin---dibekacin---tobramycin---sisomicin---gentamicin Ant2Ia
    H9C6Q1 NP_899167.1 100.00 streptogramin---macrolide---linezolid---phenicol---lincosamide cfrA
    A0A0K1YX68 AAP70012.1 100.00 beta_lactam OXA-48
    A0A109PQL3 AGK40892.1 100.00 beta_lactam TEM-207
    A0A141RLZ8 CAI76927.1 100.00 beta_lactam SHV-12
    M4TDT7 AAZ95872.1 100.00 beta_lactam CTX-M-2
    A0A127SNC5 CAB92324.1 100.00 beta_lactam TEM-68
    A0A0H4LWR1 AEZ05107.1 100.00 beta_lactam GES-19
    A0A0K1YWS6 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    D1LFH7 ACI29347.1 100.00 beta_lactam CTX-M-85
    D1LFH8 AFX60298.1 100.00 beta_lactam CTX-M-134
    F5AT71 ADE59428.1 100.00 beta_lactam CARB-2
    F5AT74 AJP67510.1 100.00 beta_lactam GES-26
    A0A097SRR7 NP_052129.1 100.00 beta_lactam TEM-116
    E5Q8Z8 ABO09821.1 100.00 beta_lactam CTX-M-79
    B2D1W0 AAK49460.1 100.00 beta_lactam OXA-35
    B2D1W1 AHC31001.1 100.00 beta_lactam OXA-246
    A0A097SRR3 NP_052129.1 100.00 beta_lactam TEM-116
    A0A097SRR2 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0N9NJE1 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0G3VHY0 AEM44654.1 100.00 beta_lactam CTX-M-109
    E9LZN2 ABY55282 100.00 cloxacillin---penicillin BL2d_oxa1
    A0A0U1YX08 ADG27454.1 100.00 beta_lactam OXA-162
    A0A066PHA3 AHL39336.1 100.00 beta_lactam ACT-24
    A0A0F4VT54 ZP_02948686 100.00 bacitracin BacA
    Q8KVS5 AAA24777 100.00 penicillin BL2a_pc
    A0A0M4HTY3 AFY98865.1 100.00 beta_lactam CTX-M-139
    L7SV23 CAA11471.1 100.00 beta_lactam IMP-1
    I6ZFU2 CAG34237.1 100.00 beta_lactam OXA-1
    R4N1F6 AEM44650.1 100.00 beta_lactam CTX-M-107
    I7AP70 CAG34237.1 100.00 beta_lactam OXA-1
    A0A0F7J021 AHA80105.1 100.00 beta_lactam ACT-20
    I7AP77 AIA58911.1 100.00 beta_lactam OXA-1
    A0A0E1L7A4 YP_001880923 100.00 macrolide MacB
    A0A0E9EAG9 ZP_04433866 100.00 bacitracin BacA
    M5JLW0 EEQ96502 100.00 bacitracin BacA
    X2LF34 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    X2LF31 ADM21467.1 100.00 beta_lactam CMY-42
    C6CXE6 ZP_02846059 100.00 bacitracin BacA
    A0A156LB93 AFU25653.1 100.00 beta_lactam ACT-15
    X2KPY5 CAA76196.1 100.00 beta_lactam DHA-1
    Q8VQ11 AAC32890.1 100.00 beta_lactam CTX-M-5
    D2KB78 ACV32635.1 100.00 beta_lactam SHV-125
    Q8VQ12 AAL86924.1 100.00 beta_lactam CTX-M-22
    D2KB73 CEA29750.1 100.00 beta_lactam SHV-187
    Q19RL9 ADI55020.1 100.00 beta_lactam PER-1
    D2KB77 ACV32635.1 100.00 beta_lactam SHV-125
    W5S9S1 CAA66659.1 100.00 beta_lactam TEM-42
    D2KB75 ACV32635.1 100.00 beta_lactam SHV-125
    M9TFP9 AAW82909 100.00 cephalosporin BL1_ec
    L0HRT9 AFY98865.1 100.00 beta_lactam CTX-M-139
    A0A0G3EIQ9 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A076V500 BAD16611.1 100.00 beta_lactam CTX-M-36
    A0A141RGD4 YP_859822 100.00 cephalosporin BL1_ec
    K7DXC3 ABF67771 100.00 streptomycin---spectinomycin Ant3Ia
    S5VEI0 AAA24777 100.00 penicillin BL2a_pc
    B8ZI10 AAL78275.1 100.00 beta_lactam OXY-1-3
    B8ZI11 CAB42614.1 100.00 beta_lactam OXY-2-4
    K9M2D2 AAA25979 100.00 penicillin---carbenicillin BL2c_pse1
    G0ZS43 NP_052129.1 100.00 beta_lactam TEM-116
    C8C0A0 AIA58911.1 100.00 beta_lactam OXA-1
    A0A076V5G4 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A0S3B061 AAF04388.1 100.00 beta_lactam CTX-M-8
    A0A0S3B062 AAF04388.1 100.00 beta_lactam CTX-M-8
    A0A0H4FVE8 ACV32635.1 100.00 beta_lactam SHV-125
    A0A023VFR5 CAA39164.1 100.00 beta_lactam SHV-5
    C0L7J1 YP_008532725.1 100.00 beta_lactam KPC-2
    C7EDS8 AEM44654.1 100.00 beta_lactam CTX-M-109
    C7EDS7 AAT35601 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A127SIB3 YP_859822 100.00 cephalosporin BL1_ec
    A0A127SK71 ACF32746.1 100.00 beta_lactam TEM-163
    I6ZGA4 CAD24670.1 100.00 beta_lactam TEM-30
    K7WT59 AGD91915.1 100.00 beta_lactam OXA-232
    I6ZGA0 AIT76103.1 100.00 beta_lactam DHA-17
    A0A0F6LN46 BAC61820.1 100.00 beta_lactam CARB-22
    M4QAY8 ACP18863.1 100.00 beta_lactam CTX-M-88
    R9XWY0 YP_008532725.1 100.00 beta_lactam KPC-2
    G0ZRI3 AAL05630.1 100.00 beta_lactam KPC-3
    A0A141RMK0 CAI43423.1 100.00 beta_lactam OXY-6-2
    V5NC86 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    G3JVF9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0K1L7M3 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0K1YWT1 AAL02127.1 100.00 beta_lactam CTX-M-15
    M9TFB9 YP_002806354 100.00 cephalosporin BL1_ec
    E0XCS6 AAA24777 100.00 penicillin BL2a_pc
    A0A127SEU2 ADC91899.1 100.00 beta_lactam GES-14
    X2J3X1 AIA58911.1 100.00 beta_lactam OXA-1
    A0A141RP72 NP_863283 100.00 penicillin BL2a_pc
    C1KBE6 AAA24777 100.00 penicillin BL2a_pc
    G9J624 AFA51701.1 100.00 beta_lactam CTX-M-123
    G9J625 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A127SHL9 ACT97415.1 100.00 beta_lactam CblA-1
    F4Y5N5 AAL79541.1 100.00 beta_lactam OXY-2-2
    A6FJS0 AGZ20205.1 100.00 beta_lactam TEM-205
    G9J628 AAY84742.1 100.00 beta_lactam CTX-M-42
    G9J629 AEM44650.1 100.00 beta_lactam CTX-M-107
    P37322 AAB19430.2 100.00 beta_lactam CARB-3
    A0A127SCX1 CAA80304.1 100.00 beta_lactam OXA-11
    A0A068JLF5 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0M4AZQ6 AAP70012.1 100.00 beta_lactam OXA-48
    G3G189 BAB16308.1 100.00 beta_lactam TEM-91
    A0A0G3ENA8 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A0F6QRZ2 CAA63262.1 100.00 beta_lactam CTX-M-1
    X2JNM4 ACU00153.1 100.00 beta_lactam CTX-M-114
    A0A0D5CA65 AAR99493.1 100.00 beta_lactam CTX-M-34
    A0A0P8GK39 ACF32746.1 100.00 beta_lactam TEM-163
    A0A0P8GXS0 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D3JYW4 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D3JYW3 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D3JYW2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D3JYW1 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D3JYW0 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A068F3Z2 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A127SDY7 AAS48620.1 100.00 beta_lactam VEB-3
    A0A127T585 CAA29443.1 100.00 beta_lactam OXA-2
    D2KB79 ACV32635.1 100.00 beta_lactam SHV-125
    D2XSF2 AHX39588.1 100.00 beta_lactam CTX-M-144
    Q84AE9 AAL78278.2 100.00 beta_lactam OXY-2-6
    O05197 AAB51122 100.00 tetracycline tetQ
    A0A076Q0S5 AFY98865.1 100.00 beta_lactam CTX-M-139
    F6M3K0 AAK71471.1 100.00 beta_lactam CTX-M-17
    S5RUE2 AAL86924.1 100.00 beta_lactam CTX-M-22
    E5FXA3 YP_001840873.1 100.00 beta_lactam OXA-58
    I3VF86 CAB94707.1 100.00 beta_lactam IMP-2
    A0A088FPI5 ABO64442.1 100.00 beta_lactam TEM-162
    A0A075TMT9 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0S2LLC4 AGC60012.1 100.00 beta_lactam OXA-244
    I7AC93 CAA76196.1 100.00 beta_lactam DHA-1
    A0A076Q261 ACV32635.1 100.00 beta_lactam SHV-125
    D0VDU1 ACQ99521.1 100.00 beta_lactam CTX-M-90
    A0A0D5C9C2 AAR99493.1 100.00 beta_lactam CTX-M-34
    D1YJY9 ABO64442.1 100.00 beta_lactam TEM-162
    D2KB74 ACV32635.1 100.00 beta_lactam SHV-125
    A0A127SCN5 AFU25638.1 100.00 beta_lactam CMY-83
    A0A127SCN2 YP_001715362.1 100.00 beta_lactam VEB-1
    I1V574 CAQ42480.1 100.00 beta_lactam CTX-M-77
    A0A0A7UX12 AHY20039.1 100.00 beta_lactam CTX-M-152
    D3J1C9 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0J1MT83 AAX54694.1 100.00 beta_lactam CTX-M-39
    D1LFE3 AFA51701.1 100.00 beta_lactam CTX-M-123
    D1LFE0 AFA51701.1 100.00 beta_lactam CTX-M-123
    F1AW45 AJO16047.1 100.00 beta_lactam SHV-189
    A0A0G3FFI5 AAT45018 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    X2JE79 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0C5BAM9 YP_008532725.1 100.00 beta_lactam KPC-2
    S5LUY4 AAK59385.1 100.00 beta_lactam IMP-9
    Q8KVT2 AAA24777 100.00 penicillin BL2a_pc
    A0A076VJC9 AAZ95872.1 100.00 beta_lactam CTX-M-2
    A0A141RE19 CAA88908.1 100.00 beta_lactam OXY-2-7
    A5Z0R6 BAM98935.1 100.00 beta_lactam IMP-43
    C8C0P4 CAD09800.1 100.00 beta_lactam TEM-1
    R4N1E8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0G3FFG1 AGG08693.1 100.00 beta_lactam CTX-M-136
    A0A077D2F3 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0M3THW4 AIH07017.1 100.00 beta_lactam KPC-19
    A0ESI5 AAT97115 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    A0A0P0A8X4 ABP04245.1 100.00 beta_lactam CTX-M-62
    C7S9T0 AAL02127.1 100.00 beta_lactam CTX-M-15
    X2LF49 CAA76196.1 100.00 beta_lactam DHA-1
    X2L7Y9 AIT76101.1 100.00 beta_lactam DHA-20
    X2LF40 AHM76771.1 100.00 beta_lactam ACT-17
    X2LF44 CAA76196.1 100.00 beta_lactam DHA-1
    X2KPX9 CAA62957.1 100.00 beta_lactam CMY-2
    I6ZR95 AAV70602.1 100.00 beta_lactam CTX-M-38
    A0A127SBF7 YP_001715362.1 100.00 beta_lactam VEB-1
    X2KPX5 CAA76196.1 100.00 beta_lactam DHA-1
    A0A088G9Q8 CAP60699.1 100.00 beta_lactam CMY-38
    A0A0P0FQB3 CAQ03505.1 100.00 beta_lactam SHV-100
    A0A127SGJ5 AAS20534 100.00 cloxacillin---penicillin BL2d_oxa2
    A0A145WTJ9 AHM76771.1 100.00 beta_lactam ACT-17
    I7AP60 AAO61597.1 100.00 beta_lactam CTX-M-27
    A0A0N0F5J7 YP_274443 100.00 bacitracin BacA
    A0A0D5CAC1 AAR99493.1 100.00 beta_lactam CTX-M-34
    A0A0D5CAC6 AFY98865.1 100.00 beta_lactam CTX-M-139
    A0A0S2A481 AIT76118.1 100.00 beta_lactam OXA-397
    A0A0X9U5K8 AGK40892.1 100.00 beta_lactam TEM-207
    A0A0G3EIR2 CAD70280.1 100.00 beta_lactam CTX-M-28
    B8ZI00 AAL78275.1 100.00 beta_lactam OXY-1-3
    M4Y3A1 AAG13410 100.00 cephalosporin---cabapenem---cephamycin---penicillin BL2_kpc
    U6BLW0 AAO61597.1 100.00 beta_lactam CTX-M-27
    C8C0B3 AIA58911.1 100.00 beta_lactam OXA-1
    A0A0G3B2Z3 AAF05311.2 100.00 beta_lactam CTX-M-9
    X2L7X4 CAA76196.1 100.00 beta_lactam DHA-1
    B8ZI15 CAI43417.1 100.00 beta_lactam OXY-5-1
    A0A141RBC7 AAL78278.2 100.00 beta_lactam OXY-2-6
    A0A089P9F5 AGC60012.1 100.00 beta_lactam OXA-244
    D3YMB3 AEM44650.1 100.00 beta_lactam CTX-M-107
    G9DDB6 YP_008532725.1 100.00 beta_lactam KPC-2
    D0E7H5 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A088GAJ9 BAF95849.1 100.00 beta_lactam SHV-111
    A0A0B5E494 NP_052129.1 100.00 beta_lactam TEM-116
    A0A141RBC6 CAB42614.1 100.00 beta_lactam OXY-2-4
    G3JVG1 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    G3JVG0 CAA71322.1 100.00 beta_lactam TEM-47
    G3JVG3 ACF32746.1 100.00 beta_lactam TEM-163
    G3JVG2 AAL03985.1 100.00 beta_lactam TEM-71
    G3JVG5 AAS89982.1 100.00 beta_lactam TEM-112
    G3JVG4 CAA71323.1 100.00 beta_lactam TEM-48
    G3JVG7 ACF32746.1 100.00 beta_lactam TEM-163
    G3JVG6 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A088FTT8 AAP70012.1 100.00 beta_lactam OXA-48
    A0FJ04 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0P8SGZ5 CAA38429.1 100.00 beta_lactam TEM-2
    W0ICA1 YP_001840873.1 100.00 beta_lactam OXA-58
    Q6WWY8 NP_052129.1 100.00 beta_lactam TEM-116
    A0A127T6X0 CAB42614.1 100.00 beta_lactam OXY-2-4
    Q6WWY2 AEJ08681.1 100.00 beta_lactam SHV-52
    Q6WWY3 NP_052129.1 100.00 beta_lactam TEM-116
    Q6WWY7 NP_052129.1 100.00 beta_lactam TEM-116
    Q6WWY4 NP_052129.1 100.00 beta_lactam TEM-116
    Q6WWY5 NP_052129.1 100.00 beta_lactam TEM-116
    G9J633 AFA51701.1 100.00 beta_lactam CTX-M-123
    A6FJR7 ABC96711.1 100.00 beta_lactam TEM-149
    C5IY58 AAZ95872.1 100.00 beta_lactam CTX-M-2
    G9J637 AAY84742.1 100.00 beta_lactam CTX-M-42
    G9J636 AFA51701.1 100.00 beta_lactam CTX-M-123
    G9J635 AET99223.1 100.00 beta_lactam CTX-M-117
    G9J634 AFA51701.1 100.00 beta_lactam CTX-M-123
    G9J638 AEM44650.1 100.00 beta_lactam CTX-M-107
    A0A0U0GRK3 ZP_03232649 100.00 penicillin BL2a_III
    A0A127SHK3 AEG64812.1 100.00 beta_lactam TEM-185
    A0A127T6N4 AAL78278.2 100.00 beta_lactam OXY-2-6
    A0A097JT52 YP_008532725.1 100.00 beta_lactam KPC-2
    B6SCD8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0M4CJ04 AGW25368.1 100.00 beta_lactam CTX-M-142
    B7U6G2 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    A0A0C5B5N5 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A0E3JP45 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0A7MB39 ACB71165.1 100.00 beta_lactam KPC-6
    C7E9V5 AAF05311.2 100.00 beta_lactam CTX-M-9
    A0A127SJU9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0G3XR69 AAP70012.1 100.00 beta_lactam OXA-48
    Q9F614 AAK49460.1 100.00 beta_lactam OXA-35
    A0A127SDX4 AAK14293.1 100.00 beta_lactam VEB-1a
    A0A0S2PJL6 AGW25368.1 100.00 beta_lactam CTX-M-142
    A0A0S1LIZ1 AIA58911.1 100.00 beta_lactam OXA-1
    R4MXL8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A7Y2V4 AAS20534 100.00 cloxacillin---penicillin BL2d_oxa2
    A0A127SD38 YP_002806354 100.00 cephalosporin BL1_ec
    S5RUD1 ACV32635.1 100.00 beta_lactam SHV-125
    A0A0S1RVY9 CCE73593.2 100.00 beta_lactam OXA-258
    A0A127SNE5 AGL39384.1 100.00 beta_lactam TEM-208
    A0ESG6 AHM76771.1 100.00 beta_lactam ACT-17
    U3PRI3 YP_008532725.1 100.00 beta_lactam KPC-2
    I1V581 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    I1V587 AAT70415.1 100.00 beta_lactam CTX-M-37
    I1V586 AIL54055.1 100.00 beta_lactam CTX-M-115
    A0A0K2YV35 AIA58911.1 100.00 beta_lactam OXA-1
    Q000R7 CAJ28994.1 100.00 beta_lactam ACT-2
    D1LFJ7 AAL78161.1 100.00 beta_lactam OXY-4-1
    D1LFJ6 CAI43425.1 100.00 beta_lactam OXY-6-4
    Q0Z8S4 AEZ05108.1 100.00 beta_lactam GES-20
    I1V561 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A0A7UX01 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    D1LFD5 AFA51701.1 100.00 beta_lactam CTX-M-123
    D1LFD6 AFA51701.1 100.00 beta_lactam CTX-M-123
    A0A0P8S7T5 AAK14792.1 100.00 beta_lactam TEM-88
    D1LFD3 BAF63422.1 100.00 beta_lactam CTX-M-64
    D1LFD8 AFA51701.1 100.00 beta_lactam CTX-M-123
    Q6R319 CAG28417.1 100.00 beta_lactam CTX-M-96
    K7X1Q8 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A142EAN5 YP_001102017 100.00 streptomycin---spectinomycin Ant3Ia
    A0A0E1CMP0 AAL05630.1 100.00 beta_lactam KPC-3
    I1V5A4 AAR99493.1 100.00 beta_lactam CTX-M-34
    A0A0L1NLD2 ZP_04228847 100.00 penicillin BL2a_1
    A0A0H3GXZ2 AAF72530.1 100.00 beta_lactam CTX-M-14
    Q6QWN2 AAL05630.1 100.00 beta_lactam KPC-3
    A0A0N7CPS8 AAP70012.1 100.00 beta_lactam OXA-48
    A0ESH3 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    Q6ZY57 AAU89706.1 100.00 beta_lactam LEN-16
    A0ESH5 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0ESH4 AAF72531.1 100.00 beta_lactam CTX-M-13
    R9XWY4 CAD70280.1 100.00 beta_lactam CTX-M-28
    E7CHE4 ABF47919.1 100.00 beta_lactam OXA-96
    A0ESH8 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0A0D1RA55 ZP_04121322 100.00 penicillin BL2a_1
    A0A0D6A134 CAI76927.1 100.00 beta_lactam SHV-12
    A0A0M4KTM9 AEM44654.1 100.00 beta_lactam CTX-M-109
    X2LF54 CAA76196.1 100.00 beta_lactam DHA-1
    I6ZR88 AAV70602.1 100.00 beta_lactam CTX-M-38
    R9YJE3 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0A0H4FP00 CAA71321.1 100.00 beta_lactam CTX-M-3
    Q849T1 AAL78278.2 100.00 beta_lactam OXY-2-6
    A0A060KQ21 AAP43508.1 100.00 beta_lactam CTX-M-30
    A0A0T9WUV6 CAR59921 100.00 macrolide MacB
    A0A0D5C9C8 AEM44654.1 100.00 beta_lactam CTX-M-109
    V9Z8P7 CAA66659.1 100.00 beta_lactam TEM-42
    A0A023ZTE5 AGC60012.1 100.00 beta_lactam OXA-244
    A0A0S2A492 ABF47919.1 100.00 beta_lactam OXA-96
    A0A098YKJ1 CAA66659.1 100.00 beta_lactam TEM-42
    B6RWQ3 YP_001844885.1 100.00 beta_lactam OXA-20
    A0A109YDC5 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0K0YDC0 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    D1LU76 AAK29174.1 100.00 beta_lactam VEB-2
    I2A684 AAZ95872.1 100.00 beta_lactam CTX-M-2
    D1LU78 AAG45720.1 100.00 beta_lactam OXA-10
    A0A076V5I3 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A088SAN6 CAG28417.1 100.00 beta_lactam CTX-M-96
    A0A076V5I9 BAD16611.1 100.00 beta_lactam CTX-M-36
    D3Y3G2 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    A0A141RKZ2 CAI43422.1 100.00 beta_lactam OXY-6-1
    U6BLV2 AFQ23961.1 100.00 beta_lactam SHV-155
    I7ACB1 YP_008532725.1 100.00 beta_lactam KPC-2
    C0L014 CAA71699.2 100.00 beta_lactam OXA-21
    D0E7I9 CAQ42486.1 100.00 beta_lactam CTX-M-76
    A0A0N9PAV0 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    X2JG68 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D0E7I4 AAC32890.1 100.00 beta_lactam CTX-M-5
    D0E7I6 AAC32890.1 100.00 beta_lactam CTX-M-5
    F8UV52 YP_008532725.1 100.00 beta_lactam KPC-2
    H9NAM7 AAA23441 100.00 cephalosporin BL1_ec
    G9I575 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0F6QPT1 ABN09669.1 100.00 beta_lactam CTX-M-61
    A0A0F6QPT4 AJO16044.1 100.00 beta_lactam TEM-214
    R9RMD3 CAC85660.1 100.00 beta_lactam TEM-93
    A0A0F7NFI3 AET99223.1 100.00 beta_lactam CTX-M-117
    A0A024A3C8 ABP04245.1 100.00 beta_lactam CTX-M-62
    A0A089MZE7 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0D5CRD0 CAA63262.1 100.00 beta_lactam CTX-M-1
    A0A0E1M940 YP_002806354 100.00 cephalosporin BL1_ec
    C6GJY7 AAS20534 100.00 cloxacillin---penicillin BL2d_oxa2
    A0A0F7CWL2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    C8CE52 ADA79630.1 100.00 beta_lactam TEM-171
    D5L080 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D5L087 CAC67290.1 100.00 beta_lactam TEM-95
    D5L086 AAL03985.1 100.00 beta_lactam TEM-71
    X2JE75 AGK82336.1 100.00 beta_lactam TEM-206
    A0A0H4FPZ2 ABI34705.1 100.00 beta_lactam CTX-M-55
    D5L088 ACF32746.1 100.00 beta_lactam TEM-163
    W1EFB6 AIA58911.1 100.00 beta_lactam OXA-1
    U5LM92 CAA11471.1 100.00 beta_lactam IMP-1
    A0A127SDI7 CAA80304.1 100.00 beta_lactam OXA-11
    S4VCM1 AAG45720.1 100.00 beta_lactam OXA-10
    A0A127T4T0 CAA29443.1 100.00 beta_lactam OXA-2
    A0A0B5FPE2 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A0N9JNV6 AGC60012.1 100.00 beta_lactam OXA-244
    A0A0B5FRQ7 ACP18863.1 100.00 beta_lactam CTX-M-88
    A0A0B5FRQ2 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0P0G0J5 CAI30650.2 100.00 beta_lactam SHV-61
    A0A0C5B9Y7 YP_008532725.1 100.00 beta_lactam KPC-2
    Q2LD93 AFX60298.1 100.00 beta_lactam CTX-M-134
    C1KLB4 YP_008532725.1 100.00 beta_lactam KPC-2
    H9NAQ8 AIA58911.1 100.00 beta_lactam OXA-1
    A0A076Q0U7 CAI76927.1 100.00 beta_lactam SHV-12
    G5DDP7 CAI76927.1 100.00 beta_lactam SHV-12
    A0A0S1LIW4 CAG34237.1 100.00 beta_lactam OXA-1
    X2D869 CAG34237.1 100.00 beta_lactam OXA-1
    A0A0S2LLE6 AAP70012.1 100.00 beta_lactam OXA-48
    A0A141RAT9 AJO16044.1 100.00 beta_lactam TEM-214
    E1IZG0 CAC85660.1 100.00 beta_lactam TEM-93
    X2KNQ1 CAA76196.1 100.00 beta_lactam DHA-1
    A0A098CN06 AFQ23961.1 100.00 beta_lactam SHV-155
    X2KNQ6 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0P8S5A6 CAD09800.1 100.00 beta_lactam TEM-1
    I7AC52 CAA63262.1 100.00 beta_lactam CTX-M-1
    C7E642 ACV92002.1 100.00 beta_lactam CTX-M-71
    G9J626 AFA51701.1 100.00 beta_lactam CTX-M-123
    G9J627 AAY84742.1 100.00 beta_lactam CTX-M-42
    V0Y7B6 AAV83795.1 100.00 beta_lactam TEM-136
    M9EGD6 B7NJS0 100.00 bacitracin BacA
    I7ABZ2 CAG34237.1 100.00 beta_lactam OXA-1
    G9J621 AAK55534.1 100.00 beta_lactam CTX-M-19
    A0A0H4FL95 AIA58911.1 100.00 beta_lactam OXA-1
    A0A0X9RS12 AGK40892.1 100.00 beta_lactam TEM-207
    I7AKM4 AAF36719.1 100.00 beta_lactam SHV-26
    Q6R320 ACS32294.1 100.00 beta_lactam CTX-M-75
    Q6R321 AAF05311.2 100.00 beta_lactam CTX-M-9
    Q2FC34 ZP_03614478 100.00 penicillin BL2a_pc
    A0A125T4T8 ABN69105.1 100.00 beta_lactam CTX-M-65
    A0A0K1LXP4 BAM62793.1 100.00 beta_lactam IMP-42
    A5J129 CAA37052.1 100.00 beta_lactam ROB-1
    A0FD75 AAK71471.1 100.00 beta_lactam CTX-M-17
    A0A060V7J7 ABF67771 100.00 streptomycin---spectinomycin Ant3Ia
    U3PSE1 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A0H4IUV1 ACV32635.1 100.00 beta_lactam SHV-125
    I6ZRI2 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0P0A8Z9 AIS67613.1 100.00 beta_lactam CTX-M-157
    A0A0F6T744 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0F6T747 AJO16044.1 100.00 beta_lactam TEM-214
    D1LFI9 AAF72531.1 100.00 beta_lactam CTX-M-13
    A0A097DEN5 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0A127SLF5 ACT97415.1 100.00 beta_lactam CblA-1
    A0A141RGX0 CAQ03505.1 100.00 beta_lactam SHV-100
    A0A0H3GVS2 BAH65231 100.00 bacitracin BacA
    A0ESE7 CAA76196.1 100.00 beta_lactam DHA-1
    A0ESE2 AAD37413.1 100.00 beta_lactam SHV-2
    A0A0B7GL65 CAA66730.1 100.00 beta_lactam SHV-2A
    T2BQ77 AGW25367.1 100.00 beta_lactam TEM-209
    U5LPP9 ACM91559.1 100.00 beta_lactam KPC-9
    I6ZR77 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A098YG59 AJU57235.1 100.00 beta_lactam CTX-M-160
    L0L630 NP_052129.1 100.00 beta_lactam TEM-116
    G3KL10 ACV32635.1 100.00 beta_lactam SHV-125
    D7ZS51 YP_002806354 100.00 cephalosporin BL1_ec
    Q48388 AAC72362.1 100.00 beta_lactam TEM-10
    B6RWP9 YP_001144149 100.00 chloramphenicol catA13
    X2LDC6 AEM44650.1 100.00 beta_lactam CTX-M-107
    A0A0X8WPL8 ABM67078.1 100.00 beta_lactam IMP-19
    R8LT43 ZP_04228022 100.00 penicillin BL2a_III
    D3Y3F9 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    D3Y3F8 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    A0A060KZW7 CAA71321.1 100.00 beta_lactam CTX-M-3
    A0A076V867 AEM44654.1 100.00 beta_lactam CTX-M-109
    X5C4D2 AAT68658.1 100.00 beta_lactam CTX-M-10
    A0A023ZRV0 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0K1P7U7 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0N7C1D4 CAB37325.2 100.00 beta_lactam SHV-15
    A0A0U2SP59 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0A0QZP3 AAF72530.1 100.00 beta_lactam CTX-M-14
    Q5W2A0 CAA63714.1 100.00 beta_lactam PER-2
    A0A0A0QZP6 CAA63262.1 100.00 beta_lactam CTX-M-1
    A0A0F7NKV1 AAR05437 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A088SAA4 ABB72225.1 100.00 beta_lactam CTX-M-53
    A0A0S2IHQ0 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0M1ZD92 ZP_03984017 100.00 bacitracin BacA
    D2Z1F2 AAF04388.1 100.00 beta_lactam CTX-M-8
    A0A0B5G0Z0 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0U3IBN4 YP_001840873.1 100.00 beta_lactam OXA-58
    B1PDJ7 CAA63262.1 100.00 beta_lactam CTX-M-1
    K9MYS0 AAX33294 100.00 cephalosporin---monobactam---penicillin BL2be_shv2
    H3KU52 CAS10898 100.00 bacitracin BacA
    A0A141RF00 AFN82060.1 100.00 beta_lactam SHV-33
    A0A0F7NFH7 AAY84742.1 100.00 beta_lactam CTX-M-42
    C7DTS0 ACY01749.1 100.00 beta_lactam IMP-26
    C7F8X2 AAR99493.1 100.00 beta_lactam CTX-M-34
    C7F8X3 AEM44654.1 100.00 beta_lactam CTX-M-109
    C7F8X0 AGW25368.1 100.00 beta_lactam CTX-M-142
    C7F8X6 AFY98865.1 100.00 beta_lactam CTX-M-139
    C7F8X7 AEM44650.1 100.00 beta_lactam CTX-M-107
    C7F8X4 AAL86924.1 100.00 beta_lactam CTX-M-22
    D7SBW8 AAK29174.1 100.00 beta_lactam VEB-2
    D5TKR8 ZP_04273511 100.00 penicillin BL2a_III
    X2JE66 AEM44654.1 100.00 beta_lactam CTX-M-109
    D5L074 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    D5L075 CAC85660.1 100.00 beta_lactam TEM-93
    A0A0H3W0X4 NP_863283 100.00 penicillin BL2a_pc
    D5L073 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    M9Y002 YP_008532725.1 100.00 beta_lactam KPC-2
    A0SXM1 AAK17194.1 100.00 beta_lactam TEM-63
    A0A0A0Q9J7 CAJ28994.1 100.00 beta_lactam ACT-2
    A0A127SFY0 AGZ20169.1 100.00 beta_lactam CMY-108
    A7U967 ACV92002.1 100.00 beta_lactam CTX-M-71
    A0A0Y2CSL2 CAC16665 100.00 penicillin BL2a_pc
    Q1HPG5 AFK08538.1 100.00 beta_lactam CMY-71
    A0A127SB51 AAK29174.1 100.00 beta_lactam VEB-2
    Q8RTG0 AAK71471.1 100.00 beta_lactam CTX-M-17
    V5NA97 ACP18863.1 100.00 beta_lactam CTX-M-88
    A0A088SFT5 AAG45720.1 100.00 beta_lactam OXA-10
    U3N9Z4 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0P8H1N7 ACF32746.1 100.00 beta_lactam TEM-163
    G5DDQ5 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0S1LIV7 AIA58911.1 100.00 beta_lactam OXA-1
    A0A0S3NK71 ZP_02902679 100.00 fosfomycin---deoxycholate mdtG
    A0A0B5G280 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0B5G286 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0P8HG85 CAA74912.2 100.00 beta_lactam TEM-17
    A0A076Q251 AFY98865.1 100.00 beta_lactam CTX-M-139
    A0A0M1XWF3 AFU35063.1 100.00 aminoglycoside ANT(9)-Ia
    A0A0W2JXZ1 ZP_02433069 100.00 chloramphenicol catA1
    G8FPM5 AAP22736.1 100.00 beta_lactam CTX-M-29
    A0A088FSY6 CAA66659.1 100.00 beta_lactam TEM-42
    Q2HYC9 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    Q2HYC8 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    J9XYK1 ABO09821.1 100.00 beta_lactam CTX-M-79
    X2KW09 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    D1LFF4 AHX39588.1 100.00 beta_lactam CTX-M-144
    C4P263 ADA79630.1 100.00 beta_lactam TEM-171
    D1LFF1 ACV92002.1 100.00 beta_lactam CTX-M-71
    A0A0H4J428 ACV32635.1 100.00 beta_lactam SHV-125
    I6V7P5 CAA66659.1 100.00 beta_lactam TEM-42
    M9Z2R8 CAJ47133.2 100.00 beta_lactam SHV-78
    Q9F9K0 AFQ23961.1 100.00 beta_lactam SHV-155
    A0A073VJ22 AAS89982.1 100.00 beta_lactam TEM-112
    A0ESE6 CAA66730.1 100.00 beta_lactam SHV-2A
    C7FF14 AGK82336.1 100.00 beta_lactam TEM-206
    C8C0S7 AHL39330.1 100.00 beta_lactam CMY-105
    L7T277 ACQ82806.1 100.00 beta_lactam PDC-2
    A0A0F7LF42 CAA38429.1 100.00 beta_lactam TEM-2
    I6V8K7 AAK13078.1 100.00 beta_lactam IMP-4
    A0A0P8R9R6 AGW25367.1 100.00 beta_lactam TEM-209
    A0A0K2FRH6 ACV32635.1 100.00 beta_lactam SHV-125
    A0ESE3 CAI76927.1 100.00 beta_lactam SHV-12
    A0ESJ9 AFQ23961.1 100.00 beta_lactam SHV-155
    A0A125QZY9 AAA23441 100.00 cephalosporin BL1_ec
    K7Q955 CAA40386.1 100.00 beta_lactam cphA7
    A0A0T9WH87 CAR59921 100.00 macrolide MacB
    A0A0G3EQ45 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A097DEM5 CAA71321.1 100.00 beta_lactam CTX-M-3
    A0A127SVW2 ACT97415.1 100.00 beta_lactam CblA-1
    A0A0F7NIY1 AAY84742.1 100.00 beta_lactam CTX-M-42
    A0A0H4FLA6 BAF63422.1 100.00 beta_lactam CTX-M-64
    A0A0H4FLA0 AAF05311.2 100.00 beta_lactam CTX-M-9
    A0A109ZX53 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    I6ZR68 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0A0WG38 AIL54055.1 100.00 beta_lactam CTX-M-115
    X2KKH0 AHM76771.1 100.00 beta_lactam ACT-17
    X2KKH5 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    A0A0U2VAA7 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0F6QQN4 CAD09800.1 100.00 beta_lactam TEM-1
    A0A0P8PWQ2 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0F6QQN1 CAA63262.1 100.00 beta_lactam CTX-M-1
    H6UQJ0 CAD70280.1 100.00 beta_lactam CTX-M-28
    X2JE82 CAD09800.1 100.00 beta_lactam TEM-1
    G3KL09 AAN04884.1 100.00 beta_lactam SHV-42
    W0NWW7 YP_001840873.1 100.00 beta_lactam OXA-58
    C8C186 AAS89982.1 100.00 beta_lactam TEM-112
    A0A127S9X4 BAF95849.1 100.00 beta_lactam SHV-111
    A0A127SRQ1 AAV83795.1 100.00 beta_lactam TEM-136
    C9EF67 AFN82060.1 100.00 beta_lactam SHV-33
    C9EF66 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    C9EF69 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0FJ05 CAA76196.1 100.00 beta_lactam DHA-1
    A0A088SAP5 CAG28417.1 100.00 beta_lactam CTX-M-96
    A0A141RKX3 CAI43422.1 100.00 beta_lactam OXY-6-1
    A0A141RKX2 CAA82916.1 100.00 beta_lactam OXY-1-1
    A2TJ38 A49789 100.00 penicillin---carbenicillin BL2c_pse1
    J7K029 BAE71359 100.00 penicillin---carbenicillin BL2c_pse1
    V5NET2 AAL02127.1 100.00 beta_lactam CTX-M-15
    X2D8B4 AIA58911.1 100.00 beta_lactam OXA-1
    A0A0H4FV89 AAK71471.1 100.00 beta_lactam CTX-M-17
    B0YKG5 NP_052129.1 100.00 beta_lactam TEM-116
    A0A068F1G7 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0F7NFY4 AIS67611.1 100.00 beta_lactam CTX-M-155
    G8FU07 ACV32635.1 100.00 beta_lactam SHV-125
    G8FU08 AAW82909 100.00 cephalosporin BL1_ec
    A0A0N9ZRX0 AAY84742.1 100.00 beta_lactam CTX-M-42
    A0A0N8N3U1 ABL67017.1 100.00 beta_lactam ACT-3
    B2CNU4 CAD70280.1 100.00 beta_lactam CTX-M-28
    T1Z0M6 AEM44654.1 100.00 beta_lactam CTX-M-109
    I6Z251 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    F1BYP4 BAA78801 100.00 streptomycin---spectinomycin Ant3Ia
    A0A0K1YWP8 AAP70012.1 100.00 beta_lactam OXA-48
    V5LZU1 AEM44654.1 100.00 beta_lactam CTX-M-109
    X5CA37 AIL54055.1 100.00 beta_lactam CTX-M-115
    A7KEG2 AAP22974.1 100.00 beta_lactam GES-7
    Q9RLH0 AAS89982.1 100.00 beta_lactam TEM-112
    A0A098L612 Q5L3H7 100.00 bacitracin BacA
    X2KPS8 BAG31347 100.00 cephalosporin---ceftriaxone---cabapenem---ceftazidime---cephamycin---cefoxitin BL1_cmy2
    K8FC57 AAG45720.1 100.00 beta_lactam OXA-10
    C7F8Y0 AEM44654.1 100.00 beta_lactam CTX-M-109
    C7F8Y3 AEM44654.1 100.00 beta_lactam CTX-M-109
    C7F8Y2 AEM44650.1 100.00 beta_lactam CTX-M-107
    C7F8Y4 CAJ47138.2 100.00 beta_lactam SHV-83
    A0A0P0HQV5 AAP70012.1 100.00 beta_lactam OXA-48
    X2L6J9 AEM44650.1 100.00 beta_lactam CTX-M-107
    Q2LE68 AAL86924.1 100.00 beta_lactam CTX-M-22
    Q4F988 AHC31001.1 100.00 beta_lactam OXA-246
    A0A089N1I4 AEM44654.1 100.00 beta_lactam CTX-M-109
    Q6BCU4 CAA11471.1 100.00 beta_lactam IMP-1
    Q6BCU6 CAA11471.1 100.00 beta_lactam IMP-1
    D6PYP1 AAL02127.1 100.00 beta_lactam CTX-M-15
    A0A0B5E3B3 NP_052129.1 100.00 beta_lactam TEM-116
    A0A0G3AZ53 CAI76927.1 100.00 beta_lactam SHV-12
    B8XLK0 ABN69105.1 100.00 beta_lactam CTX-M-65
    A0A125UJL7 NP_389667 100.00 fosfomycin FosB
    A0A0F7MZB9 CAA71321.1 100.00 beta_lactam CTX-M-3
    U6BSQ3 AIA58911.1 100.00 beta_lactam OXA-1
    D9Z5D3 CAG34237.1 100.00 beta_lactam OXA-1
    Q8RTF9 AAF72531.1 100.00 beta_lactam CTX-M-13
    G3JVH0 ACF32746.1 100.00 beta_lactam TEM-163
    G3JVH1 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    G3JVH2 CAA71322.1 100.00 beta_lactam TEM-47
    Q6R8K8 CAA62957.1 100.00 beta_lactam CMY-2
    A0A141RDT5 CAI43422.1 100.00 beta_lactam OXY-6-1
    A0A150DGJ7 ZP_04073068 100.00 penicillin BL2a_1
    E7DDC9 CAA37052.1 100.00 beta_lactam ROB-1
    A0A0B5FP83 AAL86924.1 100.00 beta_lactam CTX-M-22
    G5DDR8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    G5DDR9 ACF32746.1 100.00 beta_lactam TEM-163
    E9LZN0 ACV32633.1 100.00 beta_lactam SHV-123
    G5DDR7 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A0P9CAV5 ZP_02433069 100.00 chloramphenicol catA1
    B8ZHX7 AAL78278.2 100.00 beta_lactam OXY-2-6
    A0A0B5G293 CAA71321.1 100.00 beta_lactam CTX-M-3
    A0A0A0WJ58 YP_002806354 100.00 cephalosporin BL1_ec
    I2C9E9 ABS75156 100.00 bacitracin BacA
    B8ZHX9 AAL17873.1 100.00 beta_lactam OXY-2-8
    A0A127SHF7 ACT97415.1 100.00 beta_lactam CblA-1
    B5WXV2 AAV37206.1 100.00 beta_lactam CfxA5
    B5WXV3 AAD23513.1 100.00 beta_lactam CfxA2
    A0A075M4D2 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A088FSX9 CAA66659.1 100.00 beta_lactam TEM-42
    A2TCA3 AAL02127.1 100.00 beta_lactam CTX-M-15
    M4Q8X3 NP_052129.1 100.00 beta_lactam TEM-116
    A2TCA1 AAF72530.1 100.00 beta_lactam CTX-M-14
    A2TCA0 CAA71321.1 100.00 beta_lactam CTX-M-3
    I7AC77 AAL82592.1 100.00 beta_lactam SHV-36
    C0KUK0 AGJ01153.1 100.00 beta_lactam KPC-16
    C0KUK1 AGJ01153.1 100.00 beta_lactam KPC-16
    A0A0S2ZXB9 AAP70012.1 100.00 beta_lactam OXA-48
    D1LFA9 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A077D0D6 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    I3VF88 BAM62793.1 100.00 beta_lactam IMP-42
    A0A068EYZ9 AAP70012.1 100.00 beta_lactam OXA-48
    I6UGA4 AJO16039.1 100.00 beta_lactam IMP-47
    Q84H49 AAL68923.1 100.00 beta_lactam TEM-110
    A0A089MV24 AFY98865.1 100.00 beta_lactam CTX-M-139
    A0A109PLG9 AGK40892.1 100.00 beta_lactam TEM-207
    A5A4K5 AAR10958.1 100.00 beta_lactam TEM-131
    I6ZRG5 AFQ23954.1 100.00 beta_lactam SHV-148
    I6ZRG0 AFQ23961.1 100.00 beta_lactam SHV-155
    I7APG9 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    I7APG4 CAA76196.1 100.00 beta_lactam DHA-1
    A0ESG0 CAI76927.1 100.00 beta_lactam SHV-12
    A0ESG3 AAF72531.1 100.00 beta_lactam CTX-M-13
    F8UZC6 NP_052129.1 100.00 beta_lactam TEM-116
    A0ESG7 ADY02555.1 100.00 beta_lactam CTX-M-104
    A0ESG9 AAF72531.1 100.00 beta_lactam CTX-M-13
    A0A127SBB4 AAG45720.1 100.00 beta_lactam OXA-10
    Q4JK88 AEM44650.1 100.00 beta_lactam CTX-M-107
    Q4JK89 CAC85660.1 100.00 beta_lactam TEM-93
    H2DEM4 CAA82916.1 100.00 beta_lactam OXY-1-1
    A9LM97 YP_008532725.1 100.00 beta_lactam KPC-2
    X2KKI7 CAA76196.1 100.00 beta_lactam DHA-1
    A0A0E4C3C2 AAS20534 100.00 cloxacillin---penicillin BL2d_oxa2
    A0A0E4C3C1 CAA71699.2 100.00 beta_lactam OXA-21
    A0A0N8QAC7 YP_236080 100.00 bacitracin BacA
    B8K1U0 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A127T0Z5 ABM73648 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    A0A0N7IEF1 AGK82336.1 100.00 beta_lactam TEM-206
    B6E3J9 BAE71359 100.00 penicillin---carbenicillin BL2c_pse1
    R4HIE5 CAA66659.1 100.00 beta_lactam TEM-42
    A0A144I2K8 CAI76927.1 100.00 beta_lactam SHV-12
    U5QUS9 ZP_02433069 100.00 chloramphenicol catA1
    W0G2A6 AAT45026 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    Q5G005 AAF72530.1 100.00 beta_lactam CTX-M-14
    Q5G006 AAF72531.1 100.00 beta_lactam CTX-M-13
    C8C0F9 CAD70280.1 100.00 beta_lactam CTX-M-28
    I1W6H8 AAL82592.1 100.00 beta_lactam SHV-36
    A0A0G3FDB1 BAC53608 100.00 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm
    L1PB72 AAL79549.2 100.00 beta_lactam CfxA3
    Q849R9 ACV44456.1 100.00 beta_lactam OXY-2-10
    A0A0A0QZN9 CAD70280.1 100.00 beta_lactam CTX-M-28
    L7RFF9 ABB72225.1 100.00 beta_lactam CTX-M-53
    A0A0F7Q528 BAD16611.1 100.00 beta_lactam CTX-M-36
    A0A0H4FV93 ABN69105.1 100.00 beta_lactam CTX-M-65
    A0A0A0QZN1 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0A0QZN5 CAD89606.1 100.00 beta_lactam CTX-M-32
    A0A0A1E4Y8 YP_001102017 100.00 streptomycin---spectinomycin Ant3Ia
    A0A088SAC1 CAI76927.1 100.00 beta_lactam SHV-12
    C8C086 AAL86924.1 100.00 beta_lactam CTX-M-22
    A0A0A0R1Z6 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0Q9GIQ3 ZP_04065292 100.00 penicillin BL2a_III
    C3VJV1 NP_863283 100.00 penicillin BL2a_pc
    G8FU11 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    G5DDT6 CAD70280.1 100.00 beta_lactam CTX-M-28
    K4HQJ0 ACF32746.1 100.00 beta_lactam TEM-163
    Q9AP12 CAA39164.1 100.00 beta_lactam SHV-5
    J8IDZ2 ZP_04191906 100.00 penicillin BL2a_III
    A0A0D8VQL5 YP_002411827 100.00 macrolide MacB
    Q8VNT7 AAM11666 100.00 cephalosporin BL1_ampc
    A0A127T6U5 CAP12355.2 100.00 beta_lactam OKP-B-6
    A0A0A7V3M9 AEI70324.1 100.00 beta_lactam CTX-M-116
    D0EK08 ABP04245.1 100.00 beta_lactam CTX-M-62
    D0EK04 ABC18328.3 100.00 beta_lactam CTX-M-54
    A0A127T724 CAB42614.1 100.00 beta_lactam OXY-2-4
    A0A141RMV9 CAI43423.1 100.00 beta_lactam OXY-6-2
    W0IC20 YP_001840873.1 100.00 beta_lactam OXA-58
    G3F055 YP_008532725.1 100.00 beta_lactam KPC-2
    G3F056 YP_008532725.1 100.00 beta_lactam KPC-2
    A0A076PYL4 CAI76927.1 100.00 beta_lactam SHV-12
    A0A0U3YCX3 BAM62793.1 100.00 beta_lactam IMP-42
    A0A0P0GJN8 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    F1CGM5 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A089MX25 ACV92002.1 100.00 beta_lactam CTX-M-71
    A0A075M4U8 AEM44654.1 100.00 beta_lactam CTX-M-109
    A0A0H4FPY6 ABW86620.2 100.00 beta_lactam CTX-M-80
    A0A127T615 CAB42614.1 100.00 beta_lactam OXY-2-4
    A0A089GHS5 ADI55020.1 100.00 beta_lactam PER-1
    B4YF42 CAJ47126.2 100.00 beta_lactam SHV-71
    R9UT65 AAN04884.1 100.00 beta_lactam SHV-42
    W8YE54 ABB72225.1 100.00 beta_lactam CTX-M-53
    A0A127SIY6 ACT97415.1 100.00 beta_lactam CblA-1
    A0A0M4FLH9 AAP70012.1 100.00 beta_lactam OXA-48
    A0A0G3ELF7 CAD70280.1 100.00 beta_lactam CTX-M-28
    A0A0U2UE24 CAL26907 100.00 cephalosporin---penicillin BL2b_tem1
    A0A127SQD9 YP_001144149 100.00 chloramphenicol catA13
    A0A089N0Z5 CAD09800.1 100.00 beta_lactam TEM-1
    G3JVI4 AGK82336.1 100.00 beta_lactam TEM-206
    E7DDD4 NP_052129.1 100.00 beta_lactam TEM-116

SNPs ARGs. These args passed the previous filter but were removed after double checking the validty of the genes.

    A0A077VQM5 ADJ67256 95.36 lipopeptide---peptide Staphylococcus A0A0E3CUB8 ADJ67256 61.37 lipopeptide---peptide Staphylococcus 
    A0A0P8QTG6 ACR56331 96.24 cephalosporin---monobactam---ceftazidime---penicillin BL2be_ctxm A0A111ASD8 ADJ67256 69.05 
    lipopeptide---peptide Staphylococcus P47235 YP_002344422.1 78.90 fluoroquinolone Campylobacter NP_252410.1 NP_252410.1 100 
    fluoroquinolone---polymyxin---macrolide---trimethoprim---chloramphenicol---aminocumarin---tetracycline---beta_lactam nalC CAO91624 
    AFC91898.1 95.33 beta_lactam---penicillin PBP2x A0A0X6NZE1 AAX33294 99.65 cephalosporin---monobactam---penicillin BL2be_shv2 
    A0A133QKJ2 ADJ67256 61.79 lipopeptide---peptide Staphylococcus A0A0H2HUJ1 ADJ67256 75.12 lipopeptide---peptide Staphylococcus P50074 
    CAA55486.1 61.43 fluoroquinolone Mycobacterium A0A0E1VLC6 ADJ67256 72.26 lipopeptide---peptide Staphylococcus CAC35723.1 CAC35723.1 
    100 fluoroquinolone---tetracycline---rifampin---beta_lactam---chloramphenicol Enterobacter A0A0G7GXV7 ADJ67256 61.14 
    lipopeptide---peptide Staphylococcus A0A068E542 ADJ67256 62.05 lipopeptide---peptide Staphylococcus A0A0U1MK87 ADJ67256 95.83 
    lipopeptide---peptide Staphylococcus P21558 CAA55486.1 53.49 fluoroquinolone Mycobacterium P0A4L9 CAA55486.1 50.75 fluoroquinolone 
    Mycobacterium E5CMS4 ADJ67256 69.17 lipopeptide---peptide Staphylococcus A0A0U1EF52 ADJ67256 75.24 lipopeptide---peptide 
    Staphylococcus P50075 CAA55486.1 62.97 fluoroquinolone Mycobacterium YP_006374661.1 YP_006374661.1 100 elfamycin Enterococcus 
    A0A0M3VQW9 ADJ67256 51.26 lipopeptide---peptide Staphylococcus A0A0D1JPD7 ADJ67256 98.57 lipopeptide---peptide Staphylococcus 
    ADJ67256 ADJ67256 100 lipopeptide---peptide Staphylococcus C5QS91 ADJ67256 75.60 lipopeptide---peptide Staphylococcus ALL09868 
    ALL09868 100 lipopeptide Enterococcus C4WAS1 ADJ67256 74.76 lipopeptide---peptide Staphylococcus ABW04151 AFC91898.1 95.20 
    beta_lactam---penicillin PBP2x NP_417798.1 NP_417798.1 100 elfamycin Escherichia A0A0B8R3M9 NP_465220 99.42 peptide Listeria 
    CDO13981.1 CDO13981.1 100 macrolide---polymyxin Klebsiella NP_458564.1 NP_458564.1 100 
    fluoroquinolone---tetracycline---rifampin---beta_lactam---chloramphenicol Salmonella A0A0M2NTT4 ADJ67256 74.17 lipopeptide---peptide 
    Staphylococcus G2ZAV2 NP_465220 92.37 peptide Listeria A0A0E0UWV2 NP_465220 98.73 peptide Listeria I7KN00 ADJ67256 61.95 
    lipopeptide---peptide Staphylococcus YP_002790915.1 YP_002790915.1 100 glycopeptide bleomycin NP_415372.1 NP_415372.1 100 
    nitrofuratoin Escherichia AAB39605.1 AAB39605.1 100 elfamycin Planobispora A0A0E1AHZ9 ADJ67256 99.40 lipopeptide---peptide 
    Staphylococcus A0A0A8HTJ9 ADJ67256 50.97 lipopeptide---peptide Staphylococcus AFK60084.1 AFK60084.1 100 tetracycline Enterococcus 
    A0A0E1VRK0 ADJ67256 99.40 lipopeptide---peptide Staphylococcus A0A0S2PZE3 ADJ67256 57.42 lipopeptide---peptide Staphylococcus 
    A0A0U4VVZ8 BAA78801 99.62 streptomycin---spectinomycin Ant3Ia A0A0F5X5G9 ADJ67256 58.81 lipopeptide---peptide Staphylococcus K8E4R5 
    NP_465220 55.75 peptide Listeria U5UT68 ADJ67256 75.12 lipopeptide---peptide Staphylococcus W7BH86 NP_465220 73.31 peptide Listeria 
    B9DP65 ADJ67256 60.24 lipopeptide---peptide Staphylococcus E3CJG1 Q8DWT2 61.11 peptide Streptococcus D7UU38 NP_465220 71.64 peptide 
    Listeria A0A143P8Q3 ADJ67256 59.64 lipopeptide---peptide Staphylococcus CCE36834 CCE36834 100 fosfomycin Mycobacterium CAA67349.1 
    CAA67349.1 100 elfamycin Streptomyces W7CFM8 NP_465220 73.64 peptide Listeria A0A0T9KLW6 Q8DWT2 99.41 peptide Streptococcus Q6G9J9 
    ADJ67256 99.29 lipopeptide---peptide Staphylococcus U2IIW0 Q8DWT2 61.11 peptide Streptococcus WP_010891786.1 WP_010891786.1 100 
    fluoroquinolone Ureaplasma A0A069FV86 ADJ67256 69.76 lipopeptide---peptide Staphylococcus Y5PGH3 ADJ67256 68.81 lipopeptide---peptide 
    Staphylococcus A0A069F5G8 ADJ67256 50.78 lipopeptide---peptide Staphylococcus G5JLD2 ADJ67256 84.88 lipopeptide---peptide 
    Staphylococcus YP_002741884 AFC91898.1 90.13 beta_lactam---penicillin PBP2x CAA55486.1 CAA55486.1 100 fluoroquinolone Mycobacterium 
    A0A145P7E2 AHL39338.1 99.48 beta_lactam ACT-25 A0A0W3BDD4 ADJ67256 69.76 lipopeptide---peptide Staphylococcus Q49XF5 ADJ67256 61.26 
    lipopeptide---peptide Staphylococcus A0A156KT61 ZP_03281873 96.90 macrolide MacB M4K1L4 YP_002382193 59.32 macrolide MacB A0A070UCU2 
    B6I9E2 99.75 norfloxacin---enoxacin MdtH A0A155RYZ0 ZP_03281873 97.37 macrolide MacB A0A0C2T971 YP_001487997 99.29 bacitracin BacA 
    A0A145ECJ5 AAC75314.1 84.47 polymyxin PmrF A0A073W830 YP_002848913 98.02 fosfomycin---deoxycholate mdtG U7PEY1 ADJ67256 61.79 
    lipopeptide---peptide Staphylococcus A0A0P8SN42 ZP_03281873 96.28 macrolide MacB A0A0H1UJL4 Q8DWT2 99.29 peptide Streptococcus 
    A0A155YT88 ZP_03281873 96.59 macrolide MacB I3AII4 YP_001480516 97.45 bacitracin BacA A0A0U5Q8R0 ADJ67256 71.55 lipopeptide---peptide 
    Staphylococcus A0A157YW88 AGZ20169.1 79.27 beta_lactam CMY-108 V2IDH5 AAC75314.1 87.74 polymyxin PmrF A0A155PAJ9 AHA80105.1 99.21 
    beta_lactam ACT-20 A0A0E0YZV0 AAC75314.1 75.93 polymyxin PmrF A0A155JTR5 ZP_03281873 97.52 macrolide MacB A0A156CZK7 ZP_03281873 
    97.68 macrolide MacB A0A0H1J590 ZP_03949893 56.20 bacitracin BacA A0A154WM98 ZP_03281873 97.68 macrolide MacB A0A0H0CI94 YP_002150494 
    66.67 chloramphenicol catA4 AIL15701 AIL15701 100 fosfomycin Escherichia A0A155NC33 ACJ05689.1 99.74 beta_lactam ACT-5 V3KNE8 
    CAJ19615.1 99.65 beta_lactam OKP-B-7 A0A072C5K6 ABF50909.1 61.41 beta_lactam OCH-8 A0A155TS33 ZP_03281873 96.13 macrolide MacB 
    A0A144L9D5 AAC75314.1 84.62 polymyxin PmrF A0A0M1RHU5 YP_002115369 99.55 polymyxin arnA A0A069X774 YP_002806354 98.14 cephalosporin 
    BL1_ec D7ZXT8 AAC75314.1 99.07 polymyxin PmrF A0A133RK86 Q8DWT2 99.53 peptide Streptococcus D7ZXT7 ACI80986 99.39 polymyxin arnA 
    M5JFU5 YP_002315561 97.80 bacitracin BacA A0A0M1DRL6 ADQ43421.1 83.15 aminoglycoside APH(3')-IIc A0A0E0VNP2 ADJ67256 95.83 
    lipopeptide---peptide Staphylococcus A0A0S2EYJ3 ADJ67256 50.97 lipopeptide---peptide Staphylococcus NP_418486.1 NP_418486.1 100 
    fluoroquinolone---tetracycline---rifampin---beta_lactam---chloramphenicol Escherichia A0A156QG69 YP_002850217 78.33 polymyxin arnA 
    ABJ55333 AFC91898.1 95.20 beta_lactam---penicillin PBP2x A0A145BH70 AHM76777.1 98.95 beta_lactam ACT-18 A0A0E1EKF0 Q8DWT2 99.65 
    peptide Streptococcus A0A0W2JX83 ZP_03281873 96.75 macrolide MacB A0A0J0DEV8 ZP_03281873 96.28 macrolide MacB A0A0U8LBX4 AAK15701 
    98.41 cephalosporin BL1_sm A0A157D5T6 YP_002850217 78.03 polymyxin arnA A0A0G9GGK3 ZP_03945515 99.28 bacitracin BacA A0A156AFV6 
    AHM76777.1 99.48 beta_lactam ACT-18 AAP46614 AAP46614 100 penicillin pbp2b AAP46617 AAP46617 100 penicillin pbp2b AAP46611 AAP46611 
    100 penicillin pbp2b AAP46613 AAP46613 100 penicillin pbp2b AAP46619 AAP46619 100 penicillin pbp2b S2Y3C3 ADJ67256 51.38 
    lipopeptide---peptide Staphylococcus A0A0N0NUN7 YP_001089504 92.93 bacitracin BacA P0A0K8 CAA55486.1 52.65 fluoroquinolone 
    Mycobacterium R9CQ13 CAA65601.1 87.90 beta_lactam BlaB A0A0F5KQ47 YP_078703 83.77 fosfomycin FosB A0A0F7P1Z1 CAA65601.1 99.60 
    beta_lactam BlaB A0A0C1PR75 AAQ08905.1 97.93 beta_lactam OXA-60 A0A0G3PYI7 ZP_03837591 79.03 polymyxin arnA H0DHN2 ADJ67256 57.59 
    lipopeptide---peptide Staphylococcus A0A029N1S3 A7ZP73 99.55 peptide---polymyxin arnA A0A155M9H4 AAL05857 99.74 cephalosporin 
    BL1_ampc A0A145RM34 ZP_03281873 97.68 macrolide MacB I7KQH1 YP_302016 90.58 fosfomycin FosB A0A157MNP8 YP_002150494 63.04 
    chloramphenicol catA4 M8XNL5 YP_002806354 98.41 cephalosporin BL1_ec I3AIV2 AAC75314.1 75.48 polymyxin PmrF A0A0E0YW25 YP_002931998 
    96.32 bacitracin BacA A0A156XXU6 YP_002150494 68.12 chloramphenicol catA4 A0A0A5JLW2 ACH58991.1 54.13 beta_lactam LRA-13 A0A0J6KIT2 
    AAF86642.1 91.50 glycopeptide vanSC A0A144JAR3 YP_002150494 67.15 chloramphenicol catA4 A0A0D0SS22 ADJ67256 59.05 
    lipopeptide---peptide Staphylococcus A0A0B3XKL5 AAC75314.1 72.70 polymyxin PmrF A0A0H1UBI6 Q8DWT2 99.41 peptide Streptococcus 
    A0A151A5Q5 ADJ67256 59.19 lipopeptide---peptide Staphylococcus A0A091BSW5 ZP_04433866 58.82 bacitracin BacA A0A146GHF6 NP_465220 
    99.40 peptide Listeria A0A144KAX3 AAC75314.1 84.92 polymyxin PmrF H1G7Q2 NP_465220 95.95 peptide Listeria A0A155FG36 YP_002850217 
    77.73 polymyxin arnA A0A157G5N1 AHA80105.1 98.43 beta_lactam ACT-20 K1EY00 CAA79968.1 99.68 beta_lactam PER-1 A0A122KGQ4 AAA24777 
    99.52 penicillin BL2a_pc A0A0T7BGW6 ACH58980.1 62.80 beta_lactam LRA-1 A0A077UML1 ADJ67256 94.76 lipopeptide---peptide Staphylococcus 
    A0A0M8NI77 AAR20595.1 99.35 beta_lactam BLA1 A0A0M2W4B3 AAK15701 96.56 cephalosporin BL1_sm A0A0D1AWT3 B7LM76 78.42 
    peptide---polymyxin arnA A0A0J9FB08 CAA30695.1 60.00 chloramphenicol catIII A0A133Q3L9 ADJ67256 71.43 lipopeptide---peptide 
    Staphylococcus A0A144YWS0 YP_002850217 77.58 polymyxin arnA A0A0F6EF73 Q83QT8 99.81 peptide---polymyxin arnA A0A0F6EF71 AAC75314.1 
    99.69 polymyxin PmrF V2IE10 YP_002242988 95.99 macrolide MacB A0A156TXK8 ZP_03284557 96.70 bacitracin BacA AAP46578 AAP46578 100 
    penicillin pbp2b A0A156T2E6 AAC75314.1 85.09 polymyxin PmrF A0AJE9 NP_465220 96.18 peptide Listeria AAP46577 AAP46577 100 penicillin 
    pbp2b BAA82230.1 BAA82230.1 100 glycopeptide bleomycin A0A157C6C9 YP_002150494 65.00 chloramphenicol catA4 A0A0G3RA29 AAC75314.1 
    84.21 polymyxin PmrF EDK78328 AFC91898.1 93.87 beta_lactam---penicillin PBP2x NP_414997.1 NP_414997.1 100 
    fluoroquinolone---tetracycline---rifampin---beta_lactam---chloramphenicol Escherichia A0A0H3SL22 NP_459919 99.85 macrolide MacB 
    CAD70592 CAD70592 100 penicillin pbp2b D0MBV8 AIL92328.1 89.05 beta_lactam CARB-19 A0A0D6UF88 AAV85982.1 59.63 macrolide macB 
    AAP46635 AAP46635 100 penicillin pbp2b A0A073VI38 YP_001451669 84.48 chloramphenicol MdtL A0A157HQA3 YP_002850217 78.03 polymyxin 
    arnA E8SGZ2 ADJ67256 51.39 lipopeptide---peptide Staphylococcus A0A0M4MM10 ZP_03837411 64.51 macrolide MacB V2N412 YP_002115369 99.70 
    polymyxin arnA A0A0P8SA15 ZP_03281873 96.59 macrolide MacB K0TSA6 ADJ67256 59.24 lipopeptide---peptide Staphylococcus A0A155LDR2 
    YP_002850217 78.48 polymyxin arnA U1EF60 ADJ67256 59.36 lipopeptide---peptide Staphylococcus A0A0E1ESL7 AAV85982.1 52.88 macrolide 
    macB CAA34412 AFC91898.1 95.33 beta_lactam---penicillin PBP2x A0A0K0HL40 YP_001176128 51.00 macrolide MacB A0A0J8GK12 NP_465220 72.49 
    peptide Listeria

